In [10]:
# ============================================================
#  Improved Predictive-Policing | Google Colab script
#  Enhanced with panel data structure, temporal validation, and explainability
#  [FIXED VERSION WITH IMPROVED ERROR HANDLING]
# ============================================================

# ---------- 1. imports & configuration ---------------
import os, json, warnings, requests, calendar
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import yaml  # Add this import for saving conda environments

# Machine learning models - focused selection for small data
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
import matplotlib.pyplot as plt
import seaborn as sns

# Spatial visualization
try:
    import folium
    from folium.plugins import HeatMap
    FOLIUM_AVAILABLE = True
except ImportError:
    print("Folium not available. Install with: pip install folium")
    FOLIUM_AVAILABLE = False

# Try to import optional libraries
try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    from statsmodels.regression.mixed_linear_model import MixedLM
    MIXED_MODELS_AVAILABLE = True
except ImportError:
    print("Statsmodels not available. Install with: pip install statsmodels")
    MIXED_MODELS_AVAILABLE = False

try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    print("Optuna not available. Install with: pip install optuna")
    OPTUNA_AVAILABLE = False

# SHAP for model explainability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    print("SHAP not available. Install with: pip install shap")
    SHAP_AVAILABLE = False

# LIME for local explanations
try:
    import lime
    from lime import lime_tabular
    LIME_AVAILABLE = True
except ImportError:
    print("LIME not available. Install with: pip install lime")
    LIME_AVAILABLE = False

# Fairness libraries - improved error handling
try:
    from fairlearn.metrics import MetricFrame
    from fairlearn.reductions import Reweighing, GridSearch, ExponentiatedGradient
    from fairlearn.postprocessing import ThresholdOptimizer
    FAIRLEARN_AVAILABLE = True
    print("Fairlearn available - fairness analysis will be performed")
except ImportError:
    print("Fairlearn not available. Install with: pip install fairlearn")
    FAIRLEARN_AVAILABLE = False

try:
    import mlflow
    MLFLOW_AVAILABLE = True
except ImportError:
    print("MLflow not available. Install with: pip install mlflow")
    MLFLOW_AVAILABLE = False

# Suppress warnings
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

# ---------- 2. configuration ---------------------------------------
# User settings
TIME_RANGE = [2022, 2023, 2024]    # Years to include (train, val, test)
USE_CRIME_RATE = True              # Use per-capita crime rate instead of counts
USE_PANEL_DATA = True              # Create panel data (area × month)
MAX_RECORDS = 2_000_000            # Adjust based on computing resources
BATCH_SIZE = 100_000               # For memory efficiency
N_TRIALS = 20                      # Optuna trials - reduced to save time
CV_FOLDS = 3                       # Time series cross-validation folds
RAND_STATE = 42                    # For reproducibility
MAX_CORRELATION = 0.80             # Maximum allowed correlation between features
MAX_FEATURES = 30                  # Maximum features to select for models
USE_MLFLOW = MLFLOW_AVAILABLE      # Use MLflow for model tracking
APPLY_FAIRNESS = FAIRLEARN_AVAILABLE  # Apply fairness mitigation
USE_EXPLAINERS = SHAP_AVAILABLE or LIME_AVAILABLE  # Use model explainers

# Directories - auto-create in Colab
BASE_DIR = "/content/drive/MyDrive/predictive_policing"
# Create these directories if they don't exist (for Google Colab)
if not os.path.exists(BASE_DIR):
    BASE_DIR = "/content/predictive_policing"  # Fallback path for Colab

CACHE_DIR = f"{BASE_DIR}/cache"
VIZ_DIR = f"{BASE_DIR}/viz"
MODELS_DIR = f"{BASE_DIR}/models"
LOGS_DIR = f"{BASE_DIR}/logs"

# Create directories
for directory in [BASE_DIR, CACHE_DIR, VIZ_DIR, MODELS_DIR, LOGS_DIR]:
    os.makedirs(directory, exist_ok=True)

# API endpoints
CRIME_API = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
COMM_API = "https://data.cityofchicago.org/resource/igwz-8jzy.json"
CENSUS_API = "https://data.cityofchicago.org/resource/kn9c-c2s2.json"
BOUNDS_API = "https://data.cityofchicago.org/resource/bt9m-d2mf.json"

# MLflow experiment setup if available
if USE_MLFLOW and MLFLOW_AVAILABLE:
    mlflow.set_tracking_uri(f"file://{BASE_DIR}/mlruns")
    mlflow.set_experiment("predictive_policing")

# ---------- 3. helper functions --------------------------------------
def log(msg, level="info"):
    """Log messages to console and file"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_msg = f"{timestamp} | {msg}"
    print(log_msg)

    # Also save to log file
    with open(f"{LOGS_DIR}/run_{datetime.now():%Y%m%d}.log", "a") as f:
        f.write(log_msg + "\n")

# ---- Centralized NaN & Type Coercion ----
def safe_median(s: pd.Series) -> float:
    """Safely compute median of a Series, handling NaN values and conversion issues"""
    try:
        median = s.dropna().median()
        if hasattr(median, 'item'):
            return float(median.item())
        else:
            return float(median) if pd.notna(median) else 0.0
    except (TypeError, ValueError):
        return 0.0

def safe_convert(value):
    """Safely convert a value to float, handling various pandas/numpy types"""
    try:
        if hasattr(value, 'item'):
            return value.item()
        elif pd.isna(value):
            return 0.0
        else:
            return float(value)
    except (TypeError, ValueError):
        return 0.0

def impute_missing_values(df):
    """Impute missing values in the dataframe using safe operations"""
    log(f"Imputing missing values (before: {df.isna().sum().sum()} NaNs)")

    # Make a copy to avoid modifying the original
    df_copy = df.copy()

    # For numeric columns, use median imputation
    for col in df_copy.select_dtypes(include=['number']).columns:
        null_count = df_copy[col].isna().sum()
        if null_count > 0:
            median_val = safe_median(df_copy[col])
            df_copy[col] = df_copy[col].fillna(median_val)
            log(f"  Imputed {null_count} missing values in '{col}' with median={median_val:.3f}")

    # For categorical columns, use mode imputation
    for col in df_copy.select_dtypes(exclude=['number']).columns:
        null_count = df_copy[col].isna().sum()
        if null_count > 0:
            mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "UNKNOWN"
            df_copy[col] = df_copy[col].fillna(mode_val)
            log(f"  Imputed {null_count} missing values in '{col}' with mode={mode_val}")

    log(f"After imputation: {df_copy.isna().sum().sum()} NaNs remaining")
    return df_copy

def fetch_batches(api, limit, max_rows, params=None):
    """Fetch data in batches with query parameters"""
    rows, off = [], 0
    pbar = tqdm(total=min(max_rows, 10_000_000), unit="rec")

    query_params = {"$limit": limit, "$offset": off}
    if params:
        query_params.update(params)

    while off < max_rows:
        query_params["$offset"] = off
        try:
            r = requests.get(api, params=query_params, timeout=60)
            if r.status_code != 200:
                log(f"API error: status code {r.status_code}")
                break
            batch = r.json()
            if not batch:
                break
            rows.extend(batch)
            off += len(batch)
            pbar.update(len(batch))
        except requests.exceptions.RequestException as e:
            log(f"Request error: {str(e)}")
            break

    pbar.close()
    return pd.DataFrame(rows)

def check_data_quality(df, name):
    """Check for data quality issues"""
    log(f"Data quality check for {name} dataset:")
    log(f"  Shape: {df.shape}")
    log(f"  Missing values: {df.isna().sum().sum()}")
    log(f"  Columns: {', '.join(df.columns)}")

    # Check for outliers in numeric columns
    for col in df.select_dtypes(include=['number']).columns:
        if df[col].nunique() > 1:  # Skip constant columns
            try:
                q1, q3 = df[col].quantile(0.25), df[col].quantile(0.75)
                iqr = q3 - q1
                outliers = ((df[col] < (q1 - 1.5 * iqr)) | (df[col] > (q3 + 1.5 * iqr))).sum()
                if outliers > 0:
                    log(f"  Column {col}: {outliers} potential outliers detected")
            except Exception as e:
                log(f"  Error checking outliers for {col}: {str(e)}")

    return 1.0 - (df.isna().sum().sum() / (df.shape[0] * df.shape[1]))

def ensure_numeric(df, columns):
    """Ensure columns are numeric types"""
    # Make a copy to avoid modifying the original
    df_copy = df.copy()

    for col in columns:
        if col in df_copy.columns:
            try:
                # Store original count of non-null values
                orig_count = df_copy[col].notna().sum()

                # Try to convert to numeric, coercing errors to NaN
                df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

                # Count NaN values created
                nan_count = df_copy[col].isna().sum()

                # Report if values were lost
                if nan_count > (df_copy.shape[0] - orig_count):
                    log(f"  Converted {col} to numeric, {nan_count - (df_copy.shape[0] - orig_count)} values became NaN")
            except Exception as e:
                log(f"  Error converting {col} to numeric: {str(e)}")
    return df_copy

def ensure_numeric_coordinates(df, coord_columns=['latitude', 'longitude', 'center_lat', 'center_lon']):
    """Ensure coordinate columns are properly converted to numeric with appropriate error handling"""
    # Make a copy to avoid modifying the original
    df_copy = df.copy()

    for col in coord_columns:
        if col in df_copy.columns:
            # Store original count of non-null values
            orig_count = df_copy[col].notna().sum()

            # Convert to numeric, coercing errors to NaN
            df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

            # Report conversion results
            new_count = df_copy[col].notna().sum()
            if new_count < orig_count:
                log(f"  Warning: {orig_count - new_count} values in '{col}' couldn't be converted to numeric")

            # Fill NaN values with median if there are still valid values
            if df_copy[col].notna().any():
                median_val = df_copy[col].median()
                na_count = df_copy[col].isna().sum()
                if na_count > 0:
                    df_copy[col] = df_copy[col].fillna(median_val)
                    log(f"  Imputed {na_count} missing values in '{col}' with median={median_val:.4f}")
            else:
                # If all values are NaN, fill with a default value
                df_copy[col] = df_copy[col].fillna(0.0)
                log(f"  No valid values in '{col}', filled all with 0.0")

    return df_copy

def clean_object_columns(df):
    """Clean object columns that might contain unhashable types like dicts"""
    # Make a copy to avoid modifying the original
    df_copy = df.copy()

    object_cols = df_copy.select_dtypes(include=['object']).columns
    for col in object_cols:
        # Check if column contains dicts or other unhashable types
        try:
            # Test a small sample to see if there are unhashable types
            sample = df_copy[col].dropna().head(100)
            if len(sample) > 0:
                _ = pd.Categorical(sample)
        except (TypeError, ValueError):
            # If that fails, convert to string representation
            log(f"Converting unhashable values in column {col} to strings")
            df_copy[col] = df_copy[col].apply(lambda x: str(x) if x is not None else x)
    return df_copy

def standardize_community_area_columns(crime_df, comm_df, census_df=None):
    """Standardize community area column names across datasets"""
    log("Standardizing community area columns across datasets")

    # Make copies to avoid modifying originals
    crime_df_copy = crime_df.copy()
    comm_df_copy = comm_df.copy()
    census_df_copy = census_df.copy() if census_df is not None else None

    # List of possible column names for community area
    possible_area_cols = [
        'community_area', 'area', 'area_number', 'area_numbe', 'area_num',
        'area_num_1', 'community_area_number', 'community_area_id'
    ]

    # Standardize in crime data
    crime_area_col = next((col for col in possible_area_cols if col in crime_df_copy.columns), None)
    if crime_area_col is None:
        if 'block' in crime_df_copy.columns:
            crime_df_copy['community_area'] = crime_df_copy.block.str.extract(r'(\d+)', expand=False)
        elif 'district' in crime_df_copy.columns:
            crime_df_copy['community_area'] = crime_df_copy.district
        else:
            crime_df_copy['community_area'] = '1'  # Default value
    elif crime_area_col != 'community_area':
        crime_df_copy['community_area'] = crime_df_copy[crime_area_col]

    # Standardize in community data
    comm_area_col = next((col for col in possible_area_cols if col in comm_df_copy.columns), None)
    if comm_area_col is None:
        comm_df_copy['community_area'] = comm_df_copy.index.astype(str)
    elif comm_area_col != 'community_area':
        comm_df_copy['community_area'] = comm_df_copy[comm_area_col]

    # Standardize in census data if provided
    if census_df_copy is not None and not census_df_copy.empty:
        census_area_col = next((col for col in possible_area_cols if col in census_df_copy.columns), None)
        if census_area_col is None:
            census_df_copy['community_area'] = census_df_copy.index.astype(str)
        elif census_area_col != 'community_area':
            census_df_copy['community_area'] = census_df_copy[census_area_col]
        census_df_copy['community_area'] = census_df_copy['community_area'].astype(str)

    # Ensure all are string type and clean values
    crime_df_copy['community_area'] = crime_df_copy['community_area'].astype(str).str.strip()
    crime_df_copy['community_area'] = crime_df_copy['community_area'].str.extract(r'(\d+)', expand=False)

    comm_df_copy['community_area'] = comm_df_copy['community_area'].astype(str).str.strip()
    comm_df_copy['community_area'] = comm_df_copy['community_area'].str.extract(r'(\d+)', expand=False)

    if census_df_copy is not None:
        return crime_df_copy, comm_df_copy, census_df_copy
    else:
        return crime_df_copy, comm_df_copy

def extract_temporal_features(crime_df):
    """Extract temporal features including seasonality and trends"""
    log("Extracting temporal features")

    # Make a copy to avoid modifying the original
    crime_df_copy = crime_df.copy()

    if 'date' not in crime_df_copy.columns:
        log("No date column available for temporal features")
        return crime_df_copy

    # Ensure date is datetime
    crime_df_copy['date'] = pd.to_datetime(crime_df_copy['date'], errors='coerce')

    # Check for invalid dates
    invalid_dates = crime_df_copy['date'].isna().sum()
    if invalid_dates > 0:
        log(f"Warning: {invalid_dates} invalid dates found and will be excluded from temporal features")
        # Fill with a default date to avoid losing rows
        crime_df_copy['date'] = crime_df_copy['date'].fillna(pd.Timestamp('2000-01-01'))

    # Extract basic time components
    crime_df_copy['hour'] = crime_df_copy.date.dt.hour
    crime_df_copy['day'] = crime_df_copy.date.dt.day
    crime_df_copy['day_of_week'] = crime_df_copy.date.dt.dayofweek
    crime_df_copy['month'] = crime_df_copy.date.dt.month
    crime_df_copy['year'] = crime_df_copy.date.dt.year
    crime_df_copy['quarter'] = crime_df_copy.date.dt.quarter
    crime_df_copy['month_year'] = crime_df_copy.date.dt.to_period('M')

    # Time of day categories
    bins = [0, 6, 12, 18, 24]
    labels = ['Night', 'Morning', 'Afternoon', 'Evening']
    crime_df_copy['time_of_day'] = pd.cut(
        crime_df_copy['hour'],
        bins=bins,
        labels=labels,
        include_lowest=True
    )

    # Is weekend
    crime_df_copy['is_weekend'] = (crime_df_copy.day_of_week >= 5).astype(int)

    # Is holiday (simplified - just major US holidays)
    holidays = [
        # New Year's Day
        (1, 1),
        # Memorial Day (last Monday in May) - approximation
        (5, 31),
        # Independence Day
        (7, 4),
        # Labor Day (first Monday in September) - approximation
        (9, 1),
        # Thanksgiving (fourth Thursday in November) - approximation
        (11, 28),
        # Christmas
        (12, 25)
    ]

    crime_df_copy['is_holiday'] = 0
    for month, day in holidays:
        holiday_mask = (crime_df_copy.month == month) & (crime_df_copy.day == day)
        crime_df_copy.loc[holiday_mask, 'is_holiday'] = 1

    # Create season indicators
    crime_df_copy['is_winter'] = ((crime_df_copy.month == 12) | (crime_df_copy.month <= 2)).astype(int)
    crime_df_copy['is_spring'] = ((crime_df_copy.month >= 3) & (crime_df_copy.month <= 5)).astype(int)
    crime_df_copy['is_summer'] = ((crime_df_copy.month >= 6) & (crime_df_copy.month <= 8)).astype(int)
    crime_df_copy['is_fall'] = ((crime_df_copy.month >= 9) & (crime_df_copy.month <= 11)).astype(int)

    log("Created temporal features")

    return crime_df_copy

def create_spatial_features(crime_df, comm_df):
    """Create spatial features with row-by-row calculation for safety"""
    log("Creating spatial features with individual row processing")

    # Make copies
    crime_df_copy = crime_df.copy()
    comm_df_copy = comm_df.copy()

    # Initialize columns
    if 'geometry' not in comm_df_copy.columns:
        comm_df_copy['geometry'] = None
    comm_df_copy['center_lat'] = 0.0
    comm_df_copy['center_lon'] = 0.0
    crime_df_copy['dist_to_downtown'] = 0.0

    log("Created basic spatial features")

    # Calculate distance features if coordinates exist
    if all(col in crime_df_copy.columns for col in ['latitude', 'longitude']):
        try:
            # Convert coordinates to numeric
            crime_df_copy['latitude'] = pd.to_numeric(crime_df_copy['latitude'], errors='coerce')
            crime_df_copy['longitude'] = pd.to_numeric(crime_df_copy['longitude'], errors='coerce')

            # Downtown coordinates
            downtown_lat, downtown_lon = 41.8781, -87.6298

            # Create distance function that accepts scalar values
            def haversine_distance(lat, lon):
                """Calculate distance from a point to downtown"""
                # Skip invalid points
                if pd.isna(lat) or pd.isna(lon):
                    return np.nan

                # Convert to radians - one point at a time
                lat1, lon1 = np.radians(float(lat)), np.radians(float(lon))
                lat2, lon2 = np.radians(downtown_lat), np.radians(downtown_lon)

                # Haversine formula with scalar values
                dlat = lat2 - lat1
                dlon = lon2 - lon1
                a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                c = 2 * np.arcsin(np.sqrt(a))
                r = 6371  # Radius of Earth in kilometers
                return c * r

            # CRITICAL FIX: Process row by row instead of vectorized
            valid_rows = crime_df_copy.dropna(subset=['latitude', 'longitude'])
            log(f"Processing {len(valid_rows)} rows with valid coordinates")

            # Create a new Series for distances
            distances = pd.Series(index=valid_rows.index)

            # Calculate for each row individually - no vectorized operations
            for idx, row in valid_rows.iterrows():
                try:
                    dist = haversine_distance(row['latitude'], row['longitude'])
                    distances.loc[idx] = dist
                except Exception:
                    distances.loc[idx] = np.nan

            # Merge back to main DataFrame
            crime_df_copy.loc[distances.index, 'dist_to_downtown'] = distances

            log(f"Successfully calculated {distances.notna().sum()} distances to downtown")

            # Calculate community area stats
            if 'community_area' in crime_df_copy.columns:
                try:
                    # Get valid data subset
                    valid_data = crime_df_copy.dropna(subset=['dist_to_downtown', 'community_area'])

                    # Group by community area safely
                    if len(valid_data) > 0:
                        # Find min distance to downtown
                        area_distances = valid_data.groupby('community_area')['dist_to_downtown'].min()

                        # Convert to DataFrame with reset index
                        min_dist_df = area_distances.reset_index()
                        min_dist_df.columns = ['community_area', 'min_dist_to_downtown']

                        # Join safely
                        comm_df_copy = pd.merge(
                            comm_df_copy,
                            min_dist_df,
                            on='community_area',
                            how='left'
                        )

                        # Fill missing values
                        comm_df_copy['min_dist_to_downtown'] = comm_df_copy['min_dist_to_downtown'].fillna(0)

                        log("Added minimum distance to downtown by community area")

                        # Calculate centroids
                        centroid_data = valid_data.groupby('community_area').agg({
                            'latitude': 'mean',
                            'longitude': 'mean'
                        }).reset_index()

                        # Rename for clarity
                        centroid_data.rename(columns={
                            'latitude': 'center_lat',
                            'longitude': 'center_lon'
                        }, inplace=True)

                        # Join with community data
                        comm_df_copy = pd.merge(
                            comm_df_copy,
                            centroid_data,
                            on='community_area',
                            how='left'
                        )

                        # Fill missing values
                        for col in ['center_lat', 'center_lon']:
                            if col in comm_df_copy.columns:
                                comm_df_copy[col] = comm_df_copy[col].fillna(0)

                        log("Added community area centroids")
                except Exception as e:
                    log(f"Error in community area processing: {str(e)}")
                    # Ensure columns exist with defaults
                    if 'min_dist_to_downtown' not in comm_df_copy.columns:
                        comm_df_copy['min_dist_to_downtown'] = 0
        except Exception as e:
            log(f"Error creating distance features: {str(e)}")

    return crime_df_copy, comm_df_copy

def create_panel_data(crime_df, community_areas, time_range):
    """
    Create panel data structure (area × month) for time series prediction
    Returns a dataframe with rows for each community area and month
    """
    log("Creating panel data structure (area × month)")

    if 'date' not in crime_df.columns or 'community_area' not in crime_df.columns:
        log("Required columns missing for panel data creation")
        return pd.DataFrame()

    try:
        # Convert date and ensure community_area is string
        crime_df['date'] = pd.to_datetime(crime_df['date'], errors='coerce')
        crime_df['community_area'] = crime_df['community_area'].astype(str)

        # Create month-year column if not already present
        if 'month_year' not in crime_df.columns:
            crime_df['month_year'] = crime_df['date'].dt.to_period('M')

        # Filter to time range years
        crime_df['year'] = crime_df['date'].dt.year
        years_filter = crime_df['year'].isin(time_range)
        crime_filtered = crime_df[years_filter].copy()

        # Count crimes by community area and month
        crime_counts = crime_filtered.groupby(['community_area', 'month_year']).size().reset_index(name='crime_count')

        # Create a complete panel with all area-month combinations
        # Generate all months in the time range
        start_date = pd.Period(f"{min(time_range)}-01", freq='M')
        end_date = pd.Period(f"{max(time_range)}-12", freq='M')
        all_months = pd.period_range(start=start_date, end=end_date, freq='M')

        # Create cross product of all areas and all months
        panel_index = pd.MultiIndex.from_product(
            [community_areas, all_months],
            names=['community_area', 'month_year']
        )
        panel_df = pd.DataFrame(index=panel_index).reset_index()

        # Merge with crime counts
        panel_df = pd.merge(
            panel_df,
            crime_counts,
            on=['community_area', 'month_year'],
            how='left'
        )

        # Fill missing crime counts with 0
        panel_df['crime_count'] = panel_df['crime_count'].fillna(0)

        # Extract year and month from month_year for easier handling
        panel_df['year'] = panel_df['month_year'].dt.year
        panel_df['month'] = panel_df['month_year'].dt.month

        # Sort data for proper lag creation
        panel_df = panel_df.sort_values(['community_area', 'month_year'])

        # Create lag features (previous 1, 2, 3 months)
        # Group by community area and create lag features
        for lag in range(1, 4):
            panel_df[f'crime_count_lag{lag}'] = panel_df.groupby('community_area')['crime_count'].shift(lag)

        # Create rolling statistics (mean, std, min, max over last 3 months)
        # These are properly lagged to avoid leakage
        panel_df['rolling_mean_3m'] = panel_df.groupby('community_area')['crime_count'].shift(1).rolling(window=3, min_periods=1).mean()
        panel_df['rolling_std_3m'] = panel_df.groupby('community_area')['crime_count'].shift(1).rolling(window=3, min_periods=1).std().fillna(0)
        panel_df['rolling_min_3m'] = panel_df.groupby('community_area')['crime_count'].shift(1).rolling(window=3, min_periods=1).min()
        panel_df['rolling_max_3m'] = panel_df.groupby('community_area')['crime_count'].shift(1).rolling(window=3, min_periods=1).max()

        # Create year-over-year change (same month last year)
        panel_df['yoy_change'] = panel_df['crime_count'] - panel_df.groupby(['community_area', 'month'])['crime_count'].shift(12)

        # Create month-of-year indicators for seasonality
        for month in range(1, 13):
            panel_df[f'month_{month}'] = (panel_df['month'] == month).astype(int)

        # Create quarter indicators
        for quarter in range(1, 5):
            start_month = (quarter - 1) * 3 + 1
            end_month = quarter * 3
            panel_df[f'quarter_{quarter}'] = ((panel_df['month'] >= start_month) & (panel_df['month'] <= end_month)).astype(int)

        # Check for NaNs in lagged features
        lag_cols = [col for col in panel_df.columns if 'lag' in col or 'rolling' in col]
        for col in lag_cols:
            if col in panel_df.columns and panel_df[col].isna().sum() > 0:
                log(f"  Column {col} has {panel_df[col].isna().sum()} missing values")
                # Fill missing values with forward fill, then backward fill
                panel_df[col] = panel_df[col].fillna(method='ffill')
                panel_df[col] = panel_df[col].fillna(method='bfill')
                panel_df[col] = panel_df[col].fillna(0)  # Last resort, fill with 0

        log(f"Created panel dataset with shape {panel_df.shape}")
        log(f"Panel data missing values: {panel_df.isna().sum().sum()}")

        return panel_df

    except Exception as e:
        log(f"Error creating panel data: {str(e)}")
        # Create minimal panel data as fallback
        try:
            # Generate a simple panel with just community areas and months
            simple_panel = []
            for area in community_areas:
                for year in time_range:
                    for month in range(1, 13):
                        simple_panel.append({
                            'community_area': str(area),
                            'year': year,
                            'month': month,
                            'crime_count': 0  # Default value
                        })
            fallback_df = pd.DataFrame(simple_panel)
            log(f"Created fallback panel data with shape {fallback_df.shape}")
            return fallback_df
        except:
            return pd.DataFrame()

def create_crime_heatmap_safe(crime_df, comm_df, viz_dir):
    """Create a simplified folium heatmap with maximum error protection"""
    if not FOLIUM_AVAILABLE:
        log("Folium not available for heatmap visualization")
        return False

    try:
        log("Creating simplified crime heatmap with maximum safeguards")

        # Ensure we have coordinates
        if 'latitude' not in crime_df.columns or 'longitude' not in crime_df.columns:
            log("Latitude/longitude columns required for heatmap visualization")
            return False

        # Extract coordinates using numpy arrays to avoid pandas operations
        try:
            # Convert columns to numeric
            lat_values = pd.to_numeric(crime_df['latitude'], errors='coerce').dropna().values
            lon_values = pd.to_numeric(crime_df['longitude'], errors='coerce').dropna().values

            # Get only matching length of valid coordinates
            min_length = min(len(lat_values), len(lon_values))
            if min_length < 10:
                log("Insufficient valid coordinates")
                return False

            lat_values = lat_values[:min_length]
            lon_values = lon_values[:min_length]

            log(f"Extracted {min_length} valid coordinate pairs")

            # Sample if needed
            if min_length > 50000:
                indices = np.random.choice(min_length, 50000, replace=False)
                lat_values = lat_values[indices]
                lon_values = lon_values[indices]
                log(f"Sampled to 50,000 coordinates")
        except Exception as e:
            log(f"Error extracting coordinates: {str(e)}")
            return False

        # Create coordinate list manually without any string operations
        try:
            # Manually build list of coordinates
            heat_data = []
            for i in range(len(lat_values)):
                lat = float(lat_values[i])  # Explicit Python float conversion
                lon = float(lon_values[i])

                # Basic validation
                if -90 <= lat <= 90 and -180 <= lon <= 180:
                    heat_data.append([lat, lon])

            if len(heat_data) < 10:
                log("Too few valid coordinates after filtering")
                return False

            log(f"Created list with {len(heat_data)} valid coordinates")
        except Exception as e:
            log(f"Error creating coordinate list: {str(e)}")
            return False

        # Create and save map - with minimal folium operations
        try:
            # Create map with simplest possible syntax
            m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

            # Add heatmap layer directly
            HeatMap(data=heat_data).add_to(m)

            # Save map
            m.save(f"{viz_dir}/crime_heatmap.html")
            log("Successfully saved simplified heatmap")
            return True
        except Exception as e:
            log(f"Error in map creation/saving: {str(e)}, {type(e)}")

            # Debug - print sample of heat_data
            if len(heat_data) > 0:
                log(f"Sample heat_data: {heat_data[0]}, Type: {type(heat_data[0])}")
                if len(heat_data[0]) > 0:
                    log(f"Sample lat: {heat_data[0][0]}, Type: {type(heat_data[0][0])}")
            return False

    except Exception as e:
        log(f"Outer error in heatmap creation: {str(e)}")
        return False

def filter_highly_correlated_features_safe(df, target_col, threshold=0.8):
    """Handle correlation calculation with duplicate index protection"""
    log(f"Identifying features with correlation > {threshold} (safe implementation)")

    # Make a copy for imputation and reset index to avoid duplicates
    df_imputed = df.copy().reset_index(drop=True)

    # Get only numeric columns for correlation analysis
    numeric_cols = df_imputed.select_dtypes(include=['number']).columns.tolist()

    # Check if target is in numeric columns
    if target_col not in numeric_cols:
        log(f"Target column '{target_col}' is not numeric or not in dataframe")
        return df.columns.tolist()  # Return all columns as fallback

    # Initialize feature correlations and features to drop
    feature_correlations = {}
    features_to_drop = set()

    # Calculate correlation with target for each numeric feature
    for col in numeric_cols:
        if col == target_col:
            continue

        # Safely calculate correlation
        try:
            # Get non-null values in both columns
            mask = df_imputed[col].notna() & df_imputed[target_col].notna()
            if mask.sum() > 10:  # Only calculate if we have enough valid data points
                correlation = np.corrcoef(df_imputed[col][mask], df_imputed[target_col][mask])[0, 1]
                feature_correlations[col] = abs(correlation)

                # Check if highly correlated with target
                if abs(correlation) > threshold:
                    log(f"Feature highly correlated with {target_col}: {col} (corr={abs(correlation):.3f})")
                    features_to_drop.add(col)  # Drop features too correlated with target
        except Exception as e:
            log(f"  Error calculating correlation between {col} and {target_col}: {str(e)}")

    # Find high correlations between features
    for i, col1 in enumerate(numeric_cols):
        if col1 == target_col or col1 in features_to_drop:
            continue

        for j, col2 in enumerate(numeric_cols[i+1:], i+1):
            if col2 == target_col or col2 in features_to_drop:
                continue

            try:
                # Get non-null values in both columns
                mask = df_imputed[col1].notna() & df_imputed[col2].notna()
                if mask.sum() > 10:  # Only calculate if we have enough valid data points
                    correlation = abs(np.corrcoef(df_imputed[col1][mask], df_imputed[col2][mask])[0, 1])

                    if correlation > threshold:
                        # Determine which feature to keep based on correlation with target
                        if col1 in feature_correlations and col2 in feature_correlations:
                            if feature_correlations[col1] > feature_correlations[col2]:
                                features_to_drop.add(col2)
                                log(f"Excluding {col2} (corr={correlation:.3f}) in favor of {col1} (corr={feature_correlations[col1]:.3f})")
                            else:
                                features_to_drop.add(col1)
                                log(f"Excluding {col1} (corr={correlation:.3f}) in favor of {col2} (corr={feature_correlations[col2]:.3f})")
                                break  # Break inner loop since col1 is dropped
            except Exception as e:
                log(f"  Error checking correlation between {col1} and {col2}: {str(e)}")

    # Get final feature list
    features_to_keep = [col for col in df.columns if col not in features_to_drop or col == target_col]

    log(f"Excluded {len(features_to_drop)} features with high correlation")
    log(f"Keeping {len(features_to_keep)} features")

    return features_to_keep

def select_features(X, y, max_features=30):
    """Select the most important features using multiple methods"""

    log(f"Selecting up to {max_features} important features")

    # First impute any missing values
    X_imputed = X.copy()
    for col in X_imputed.columns:
        # Check data type of the column, not the DataFrame
        if pd.api.types.is_numeric_dtype(X_imputed[col]) and X_imputed[col].isna().any():
            X_imputed[col] = X_imputed[col].fillna(X_imputed[col].median())

    # Start with removing low variance features
    try:
        var_selector = VarianceThreshold(threshold=0.01)
        X_var = var_selector.fit_transform(X_imputed)
        var_features = X_imputed.columns[var_selector.get_support()].tolist()
        log(f"Selected {len(var_features)} features with variance > 0.01")

        if len(var_features) <= max_features:
            return var_features

        # If we still have too many features, use a tree-based selector
        X_var_df = X_imputed[var_features]

        # Use a HistGradientBoostingRegressor which handles missing values natively
        gb = HistGradientBoostingRegressor(max_iter=100, random_state=42)
        gb.fit(X_var_df, y)

        # Get feature importances
        if hasattr(gb, 'feature_importances_'):
            importances = gb.feature_importances_
        else:
            # IMPROVED FIX: Better fallback for feature importance
            try:
                # Create a standard GradientBoostingRegressor as fallback
                gb_fallback = GradientBoostingRegressor(n_estimators=100, random_state=42)
                gb_fallback.fit(X_var_df, y)
                importances = gb_fallback.feature_importances_
                log("Using fallback GradientBoostingRegressor for feature importances")
            except Exception as e:
                log(f"Error in fallback feature importance: {str(e)}")
                # For models without feature_importances_, assign equal importance
                importances = np.ones(X_var_df.shape[1]) / X_var_df.shape[1]

        # Select top features
        top_indices = np.argsort(importances)[::-1][:max_features]
        selected_features = [var_features[i] for i in top_indices]

        log(f"Selected {len(selected_features)} features by importance")

        return selected_features
    except Exception as e:
        log(f"Error in feature selection: {str(e)}")
        # Return all columns if there's an error
        return X.columns.tolist()[:max_features]

def perform_fairness_analysis(y_test, predictions, protected_attrs, viz_dir):
    """Perform fairness analysis using protected attributes"""
    log("Conducting fairness analysis")

    if not protected_attrs:
        log("No protected attributes available for fairness analysis")
        return {}

    fairness_results = {}

    # Check if Fairlearn is available
    fairlearn_available = False
    try:
        from fairlearn.metrics import MetricFrame
        fairlearn_available = True
    except ImportError:
        log("Fairlearn not available - using basic fairness metrics instead")

    # Define metrics - compatible with both Fairlearn and basic implementation
    metrics_dict = {
        'rmse': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error,
        'mean_prediction': lambda y_true, y_pred: np.mean(y_pred),
        'median_prediction': lambda y_true, y_pred: np.median(y_pred)
    }

    for attr_name, attr_values in protected_attrs.items():
        log(f"Fairness analysis for {attr_name}")

        # Handle quantile-based attributes consistently
        if np.issubdtype(attr_values.dtype, np.number) and attr_values.nunique() > 2:
            # For numeric attributes, create quartiles
            try:
                quartiles = pd.qcut(attr_values, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
                # Convert to string to avoid categorical dtype issues
                quartiles = quartiles.astype(str)
            except ValueError:  # Handle case with duplicate values
                quartiles = pd.qcut(attr_values, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'], duplicates='drop')
                quartiles = quartiles.astype(str)

            # Analyze by quartile
            results_by_quartile = {}

            # Iterate through quartiles
            for quartile in quartiles.unique():
                mask = quartiles == quartile
                quartile_y = y_test[mask]
                quartile_pred = predictions[mask]

                if len(quartile_y) > 0:
                    quartile_results = {}
                    for metric_name, metric_func in metrics_dict.items():
                        try:
                            quartile_results[metric_name] = metric_func(quartile_y, quartile_pred)
                        except Exception as e:
                            log(f"  Error calculating {metric_name} for {quartile}: {str(e)}")
                            quartile_results[metric_name] = np.nan

                    results_by_quartile[quartile] = quartile_results

                    # Log results
                    log(f"  {attr_name} {quartile}:")
                    for metric_name, value in quartile_results.items():
                        if not np.isnan(value):
                            log(f"    {metric_name}: {value:.3f}")

            # Calculate disparities across quartiles
            disparities = {}
            for metric_name in metrics_dict.keys():
                try:
                    metric_values = [results[metric_name] for results in results_by_quartile.values()
                                    if not np.isnan(results.get(metric_name, np.nan))]
                    if metric_values:
                        min_val = min(metric_values)
                        max_val = max(metric_values)
                        disparities[metric_name] = max_val - min_val
                        log(f"  {metric_name} disparity across quartiles: {disparities[metric_name]:.3f}")
                except Exception as e:
                    log(f"  Error calculating {metric_name} disparity: {str(e)}")

            fairness_results[attr_name] = {
                'quartile_results': results_by_quartile,
                'disparities': disparities
            }

            # Visualize by quartile
            try:
                plt.figure(figsize=(10, 6))
                metric_names = list(metrics_dict.keys())
                quartiles = list(results_by_quartile.keys())

                x = np.arange(len(metric_names))
                width = 0.2

                # Plot bars for each quartile
                for i, quartile in enumerate(quartiles):
                    values = []
                    for metric in metric_names:
                        if metric in results_by_quartile[quartile] and not np.isnan(results_by_quartile[quartile][metric]):
                            values.append(results_by_quartile[quartile][metric])
                        else:
                            values.append(0)  # Default for missing values
                    plt.bar(x + i*width, values, width, label=f'{quartile}')

                plt.xlabel('Metric')
                plt.ylabel('Value')
                plt.title(f'Fairness Analysis for {attr_name} by Quartile')
                plt.xticks(x + width*1.5, metric_names)
                plt.legend()
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/fairness_{attr_name}_quartiles.png", dpi=200, bbox_inches='tight')
                plt.close()
            except Exception as e:
                log(f"Error creating quartiles visualization: {str(e)}")

        # Handle categorical attributes (for hardship_quartile)
        elif isinstance(attr_values.dtype, pd.CategoricalDtype) or attr_values.dtype == 'object':
            # Handle categorical variables by converting to string
            cat_values = attr_values.astype(str)
            categories = sorted(cat_values.unique())

            # Analyze by category
            results_by_category = {}
            for category in categories:
                mask = cat_values == category
                cat_y = y_test[mask]
                cat_pred = predictions[mask]

                if len(cat_y) > 0:
                    cat_results = {}
                    for metric_name, metric_func in metrics_dict.items():
                        try:
                            cat_results[metric_name] = metric_func(cat_y, cat_pred)
                        except Exception as e:
                            log(f"  Error calculating {metric_name} for {category}: {str(e)}")
                            cat_results[metric_name] = np.nan

                    results_by_category[category] = cat_results

                    # Log results
                    log(f"  {attr_name}={category}:")
                    for metric_name, value in cat_results.items():
                        if not np.isnan(value):
                            log(f"    {metric_name}: {value:.3f}")

            # Calculate disparities
            disparities = {}
            for metric_name in metrics_dict.keys():
                try:
                    metric_values = [results[metric_name] for results in results_by_category.values()
                                    if not np.isnan(results.get(metric_name, np.nan))]
                    if metric_values:
                        min_val = min(metric_values)
                        max_val = max(metric_values)
                        disparities[metric_name] = max_val - min_val
                        log(f"  {metric_name} disparity: {disparities[metric_name]:.3f}")
                except Exception as e:
                    log(f"  Error calculating {metric_name} disparity: {str(e)}")

            fairness_results[attr_name] = {
                'category_results': results_by_category,
                'disparities': disparities
            }

            # Visualize
            try:
                plt.figure(figsize=(10, 6))
                metric_names = list(metrics_dict.keys())
                categories = list(results_by_category.keys())

                x = np.arange(len(metric_names))
                width = 0.8 / len(categories)

                # Plot bars for each category
                for i, category in enumerate(categories):
                    values = []
                    for metric in metric_names:
                        if metric in results_by_category[category] and not np.isnan(results_by_category[category][metric]):
                            values.append(results_by_category[category][metric])
                        else:
                            values.append(0)  # Default for missing values
                    plt.bar(x + i*width - 0.4 + width/2, values, width, label=f'{category}')

                plt.xlabel('Metric')
                plt.ylabel('Value')
                plt.title(f'Fairness Analysis for {attr_name}')
                plt.xticks(x, metric_names)
                plt.legend()
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/fairness_{attr_name}.png", dpi=200, bbox_inches='tight')
                plt.close()
            except Exception as e:
                log(f"Error creating categories visualization: {str(e)}")

        # Use Fairlearn for more detailed analysis if available
        elif fairlearn_available:
            try:
                # Convert to string to avoid dtype issues
                attr_series = pd.Series(attr_values, index=y_test.index).astype(str)

                mf = MetricFrame(
                    metrics=metrics_dict,
                    y_true=y_test,
                    y_pred=predictions,
                    sensitive_features=attr_series
                )

                # Log metrics by group
                log(f"Metrics by {attr_name} group:")
                for group, metrics_dict in mf.by_group.iterrows():
                    for metric, value in metrics_dict.items():
                        log(f"  {attr_name}={group}, {metric}: {value:.3f}")

                # Calculate disparities
                disparities = {}
                for metric in metrics_dict:
                    group_values = mf.by_group[metric]
                    if len(group_values) >= 2:
                        min_val = group_values.min()
                        max_val = group_values.max()
                        diff = max_val - min_val
                        disparities[metric] = diff
                        log(f"  {metric} disparity: {diff:.3f}")

                fairness_results[attr_name] = disparities

                # Visualize fairness metrics
                plt.figure(figsize=(10, 6))
                metric_names = list(metrics_dict.keys())
                group_values = list(mf.by_group.index)

                x = np.arange(len(metric_names))
                width = 0.35

                # Plot bars for each group
                for i, group in enumerate(group_values):
                    values = [mf.by_group.loc[group, metric] for metric in metric_names]
                    plt.bar(x + i*width, values, width, label=f'{attr_name}={group}')

                plt.xlabel('Metric')
                plt.ylabel('Value')
                plt.title(f'Fairness Analysis for {attr_name}')
                plt.xticks(x + width/2, metric_names)
                plt.legend()
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/fairness_{attr_name}.png", dpi=200, bbox_inches='tight')
                plt.close()
            except Exception as e:
                log(f"Error in Fairlearn fairness analysis: {str(e)}")
        else:
            log("Neither Fairlearn nor categorical/quantile analysis applicable. Using basic analysis.")
            # Basic approach for binary or simple attributes
            attr_str = attr_values.astype(str)
            unique_groups = sorted(attr_str.unique())

            # Basic analysis
            results_by_group = {}
            for group in unique_groups:
                mask = attr_str == group
                group_y = y_test[mask]
                group_pred = predictions[mask]

                if len(group_y) > 0:
                    group_results = {}
                    for metric_name, metric_func in metrics_dict.items():
                        try:
                            group_results[metric_name] = metric_func(group_y, group_pred)
                        except Exception as e:
                            group_results[metric_name] = np.nan
                            log(f"  Error calculating {metric_name} for group {group}: {str(e)}")

                    results_by_group[group] = group_results

                    # Log results
                    log(f"  {attr_name}={group}:")
                    for metric_name, value in group_results.items():
                        if not np.isnan(value):
                            log(f"    {metric_name}: {value:.3f}")

            # Calculate disparities
            disparities = {}
            for metric_name in metrics_dict.keys():
                try:
                    metric_values = [results.get(metric_name) for results in results_by_group.values()
                                   if not np.isnan(results.get(metric_name, np.nan))]
                    if metric_values:
                        disparities[metric_name] = max(metric_values) - min(metric_values)
                except Exception as e:
                    log(f"  Error calculating disparity for {metric_name}: {str(e)}")

            fairness_results[attr_name] = {
                'group_results': results_by_group,
                'disparities': disparities
            }

    return fairness_results

def create_shap_visualizations(model, X_test, feature_names, viz_dir):
    """Create SHAP visualizations for model explainability"""
    if not SHAP_AVAILABLE:
        log("SHAP not available for model explainability")
        return None

    try:
        log("Creating SHAP visualizations for model explainability")

        # Ensure we have no NaN values
        X_test_clean = X_test.copy()
        for col in X_test_clean.columns:
            if X_test_clean[col].isna().any():
                median_val = X_test_clean[col].median()
                X_test_clean[col] = X_test_clean[col].fillna(median_val)
                log(f"  Filled {X_test_clean[col].isna().sum()} NaN values in {col} with median")

        # Create SHAP explainer based on model type
        if hasattr(model, 'predict'):
            # For HistGradientBoostingRegressor, create a custom explainer
            if isinstance(model, HistGradientBoostingRegressor):
                log("Creating SHAP explainer for HistGradientBoostingRegressor")

                # Use a small sample for background data (faster)
                sample_size = min(100, X_test_clean.shape[0])
                background_data = X_test_clean.sample(sample_size, random_state=RAND_STATE)

                try:
                    # Try Explainer first (newer API)
                    explainer = shap.Explainer(model.predict, background_data)
                except Exception as e:
                    log(f"Error with shap.Explainer: {str(e)}. Trying KernelExplainer instead.")
                    explainer = shap.KernelExplainer(model.predict, background_data)
            elif isinstance(model, GradientBoostingRegressor):
                log("Creating TreeExplainer for GradientBoostingRegressor")
                try:
                    explainer = shap.TreeExplainer(model)
                except Exception as e:
                    log(f"Error with TreeExplainer: {str(e)}. Trying KernelExplainer instead.")
                    sample_size = min(100, X_test_clean.shape[0])
                    background_data = X_test_clean.sample(sample_size, random_state=RAND_STATE)
                    explainer = shap.KernelExplainer(model.predict, background_data)
            else:
                # For other models
                log(f"Creating default SHAP explainer for {type(model).__name__}")
                sample_size = min(100, X_test_clean.shape[0])
                background_data = X_test_clean.sample(sample_size, random_state=RAND_STATE)
                explainer = shap.KernelExplainer(model.predict, background_data)

            # Calculate SHAP values - use smaller sample for efficiency
            sample_size = min(100, X_test_clean.shape[0])
            data_sample = X_test_clean.sample(sample_size, random_state=RAND_STATE)

            try:
                # Try modern API first
                shap_values = explainer(data_sample)
                log("Generated SHAP values using modern API")
            except Exception as e:
                log(f"Error calculating SHAP values with __call__: {str(e)}. Trying legacy API.")
                try:
                    # Try legacy API as fallback
                    shap_values = explainer.shap_values(data_sample)
                    # Convert to shap.Explanation if needed
                    if not isinstance(shap_values, shap.Explanation):
                        shap_values = shap.Explanation(
                            values=np.array(shap_values),
                            base_values=np.array([explainer.expected_value] * len(data_sample)),
                            data=data_sample.values,
                            feature_names=feature_names
                        )
                    log("Generated SHAP values using legacy API")
                except Exception as e:
                    log(f"Error calculating SHAP values with legacy API: {str(e)}")
                    return None

            # Create visualizations with proper error handling
            try:
                # Create summary plot (bar)
                plt.figure(figsize=(12, 8))
                if isinstance(shap_values, shap.Explanation):
                    shap.summary_plot(shap_values, data_sample, plot_type="bar", feature_names=feature_names, show=False)
                else:
                    shap.summary_plot(shap_values, data_sample, plot_type="bar", feature_names=feature_names, show=False)
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/shap_summary_bar.png", dpi=200, bbox_inches='tight')
                plt.close()
                log("Created SHAP bar summary plot")

                # Create beeswarm plot for detailed feature impact
                plt.figure(figsize=(12, 10))
                if isinstance(shap_values, shap.Explanation):
                    shap.summary_plot(shap_values, data_sample, feature_names=feature_names, show=False)
                else:
                    shap.summary_plot(shap_values, data_sample, feature_names=feature_names, show=False)
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/shap_summary_beeswarm.png", dpi=200, bbox_inches='tight')
                plt.close()
                log("Created SHAP beeswarm plot")

                # Create dependence plots for top features
                if isinstance(shap_values, shap.Explanation):
                    vals = np.abs(shap_values.values).mean(0)
                    top_indices = np.argsort(vals)[-3:]
                else:
                    if isinstance(shap_values, list):
                        vals = np.abs(np.array(shap_values)).mean(0)
                    else:
                        vals = np.abs(shap_values).mean(0)
                    top_indices = np.argsort(vals)[-3:]

                for feature_idx in top_indices:
                    if feature_idx >= len(feature_names):
                        continue  # Skip if index out of bounds

                    feature_name = feature_names[feature_idx]
                    plt.figure(figsize=(10, 6))
                    try:
                        if isinstance(shap_values, shap.Explanation):
                            shap.dependence_plot(
                                feature_idx, shap_values.values, data_sample,
                                feature_names=feature_names, show=False
                            )
                        else:
                            shap.dependence_plot(
                                feature_idx, shap_values, data_sample,
                                feature_names=feature_names, show=False
                            )
                        plt.tight_layout()
                        plt.savefig(f"{viz_dir}/shap_dependence_{feature_name}.png", dpi=200, bbox_inches='tight')
                        plt.close()
                        log(f"Created SHAP dependence plot for {feature_name}")
                    except Exception as e:
                        log(f"Error creating dependence plot for {feature_name}: {str(e)}")
                        plt.close()

            except Exception as e:
                log(f"Error creating SHAP visualizations: {str(e)}")

            log("SHAP visualizations created successfully")
            return shap_values
        else:
            log("Model does not have predict method, cannot create SHAP visualizations")
            return None
    except Exception as e:
        log(f"Error creating SHAP visualizations: {str(e)}")
        return None

def create_lime_explanations(model, X_train, X_test, feature_names, viz_dir, num_samples=5):
    """Create LIME explanations for model interpretability with proper version handling"""
    if not LIME_AVAILABLE:
        log("LIME not available for model interpretability")
        return

    try:
        log("Creating LIME explanations for sample predictions")

        # Ensure X_train and X_test don't have NaN values
        X_train_clean = X_train.copy()
        X_test_clean = X_test.copy()

        for df in [X_train_clean, X_test_clean]:
            for col in df.columns:
                if df[col].isna().any():
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)

        # Create LIME explainer
        try:
            explainer = lime_tabular.LimeTabularExplainer(
                X_train_clean.values,
                feature_names=feature_names,
                mode='regression',
                verbose=False
            )

            # Sample a few instances to explain
            sample_indices = np.random.choice(X_test_clean.shape[0], min(num_samples, X_test_clean.shape[0]), replace=False)

            for i, idx in enumerate(sample_indices):
                try:
                    # Get explanation for this instance
                    exp = explainer.explain_instance(
                        X_test_clean.iloc[idx].values,
                        model.predict,
                        num_features=10
                    )

                    # Create a figure and save it manually - WITHOUT using ax parameter
                    # which causes compatibility issues
                    try:
                        # Try legacy method without passing ax parameter
                        fig = plt.figure(figsize=(10, 6))
                        exp.as_pyplot_figure()  # Do not pass ax parameter
                        plt.title(f"LIME Explanation for Instance {idx}")
                        plt.tight_layout()
                        plt.savefig(f"{viz_dir}/lime_explanation_{i}.png", dpi=200, bbox_inches='tight')
                        plt.close()
                    except Exception as viz_error:
                        log(f"Error visualizing LIME with pyplot: {str(viz_error)}")
                        # Fallback: Just save the explanation text
                        explanation_text = "\n".join([f"{feature}: {weight:.4f}" for feature, weight in exp.as_list()])
                        with open(f"{viz_dir}/lime_explanation_{i}.txt", "w") as f:
                            f.write(f"LIME Explanation for Instance {idx}\n\n{explanation_text}")

                    # Log the explanation details
                    log(f"LIME explanation for instance {idx}:")
                    explanation_list = exp.as_list()
                    for feature, weight in explanation_list:
                        log(f"  {feature}: {weight:.4f}")
                except Exception as e:
                    log(f"Error creating explanation for instance {idx}: {str(e)}")
                    continue

            log(f"Created LIME explanations for {len(sample_indices)} instances")
        except Exception as e:
            log(f"Error creating LIME explainer: {str(e)}")
    except Exception as e:
        log(f"Error in LIME explanation process: {str(e)}")

def convert_to_serializable(obj):
    """Convert NumPy types to Python native types for JSON serialization with improved error handling"""
    try:
        if hasattr(np, 'integer') and isinstance(obj, np.integer):
            return int(obj)
        elif hasattr(np, 'floating') and isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, (np.int8, np.int16, np.int32, np.int64,
                           np.uint8, np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif isinstance(obj, pd.Period):
            return str(obj)
        elif isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(i) for i in obj]
        elif pd.isna(obj):
            return None
        else:
            # Try str() as a last resort
            try:
                return str(obj)
            except:
                return "non-serializable"
    except Exception as e:
        # Last line of defense - return a placeholder for anything that can't be serialized
        return f"non-serializable ({type(obj).__name__})"

def plot_model_comparison(model_results, viz_dir):
    """Create visualization of model comparison metrics with improved error handling"""
    try:
        metrics = ['RMSE', 'R2', 'MAE']  # Use R2 instead of R² to avoid issues
        model_names = list(model_results.keys())

        plt.figure(figsize=(15, 5))
        for i, metric in enumerate(metrics):
            plt.subplot(1, 3, i+1)

            # Get lowercase metric name for dictionary access
            lowercase_metric = metric.lower()

            # Extract values, handling potential missing metrics
            values = []
            for model in model_names:
                if lowercase_metric in model_results[model]:
                    # Handle NaN values
                    value = model_results[model][lowercase_metric]
                    if pd.isna(value):
                        values.append(0)
                    else:
                        values.append(value)
                else:
                    values.append(0)  # Default value if metric is missing

            bars = plt.bar(model_names, values)

            # Add value labels on top of bars
            for bar in bars:
                height = bar.get_height()
                if height != 0:  # Only add labels for non-zero values
                    plt.text(bar.get_x() + bar.get_width()/2., height,
                           f'{height:.3f}',
                           ha='center', va='bottom', rotation=90 if len(model_names) > 3 else 0)

            # Use display-friendly labels
            display_metric = 'R²' if metric == 'R2' else metric
            plt.title(f'Comparison by {display_metric}')
            plt.ylabel(display_metric)
            plt.xticks(rotation=45)
            plt.tight_layout()

        plt.savefig(f"{viz_dir}/model_comparison.png", dpi=200, bbox_inches='tight')
        plt.close()

        log("Model comparison chart created successfully")
    except Exception as e:
        log(f"Error creating model comparison chart: {str(e)}")
        # Try a simpler chart as fallback
        try:
            plt.figure(figsize=(10, 6))

            # Just plot RMSE as a simple bar chart
            values = []
            for model in model_results:
                if 'rmse' in model_results[model]:
                    value = model_results[model]['rmse']
                    if pd.isna(value):
                        values.append(0)
                    else:
                        values.append(value)
                else:
                    values.append(0)

            plt.bar(model_names, values)
            plt.title('Model Comparison by RMSE')
            plt.ylabel('RMSE')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f"{viz_dir}/model_comparison_simple.png", dpi=200, bbox_inches='tight')
            plt.close()
            log("Created simplified model comparison chart as fallback")
        except Exception as e2:
            log(f"Error creating fallback comparison chart: {str(e2)}")

def plot_seasonal_patterns(panel_df, viz_dir):
    """Visualize seasonal patterns in crime data with improved error handling"""
    try:
        log("Creating seasonal patterns visualization")

        # Check if required columns exist
        if not all(col in panel_df.columns for col in ['year', 'month', 'crime_count']):
            log("Required columns missing for seasonal patterns visualization")
            return

        # Ensure data types are correct
        panel_df_clean = panel_df.copy()
        panel_df_clean['year'] = pd.to_numeric(panel_df_clean['year'], errors='coerce')
        panel_df_clean['month'] = pd.to_numeric(panel_df_clean['month'], errors='coerce')
        panel_df_clean['crime_count'] = pd.to_numeric(panel_df_clean['crime_count'], errors='coerce')

        # Drop rows with NaN values after conversion
        panel_df_clean = panel_df_clean.dropna(subset=['year', 'month', 'crime_count'])

        if len(panel_df_clean) == 0:
            log("No valid data after cleaning for seasonal patterns")
            return

        # Create monthly average crime by year
        monthly_crime = panel_df_clean.groupby(['year', 'month'])['crime_count'].mean().reset_index()

        # Pivot for better visualization
        try:
            monthly_pivot = monthly_crime.pivot(index='month', columns='year', values='crime_count')

            # Plot
            plt.figure(figsize=(12, 8))
            sns.lineplot(data=monthly_pivot)
            plt.title('Monthly Crime Patterns by Year')
            plt.xlabel('Month')
            plt.ylabel('Average Crime Count per Community Area')
            plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
            plt.legend(title='Year')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f"{viz_dir}/monthly_crime_patterns.png", dpi=200, bbox_inches='tight')
            plt.close()

            log("Created monthly crime patterns visualization")
        except Exception as e:
            log(f"Error creating monthly pivot: {str(e)}")
            # Try alternative visualization
            try:
                plt.figure(figsize=(12, 8))
                for year in sorted(monthly_crime['year'].unique()):
                    year_data = monthly_crime[monthly_crime['year'] == year]
                    plt.plot(year_data['month'], year_data['crime_count'], label=str(int(year)))
                plt.title('Monthly Crime Patterns by Year')
                plt.xlabel('Month')
                plt.ylabel('Average Crime Count per Community Area')
                plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
                plt.legend(title='Year')
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig(f"{viz_dir}/monthly_crime_patterns_alt.png", dpi=200, bbox_inches='tight')
                plt.close()
                log("Created alternative monthly crime patterns visualization")
            except Exception as e2:
                log(f"Error creating alternative monthly visualization: {str(e2)}")

        # Create day of week patterns
        if 'day_of_week' in panel_df_clean.columns:
            try:
                panel_df_clean['day_of_week'] = pd.to_numeric(panel_df_clean['day_of_week'], errors='coerce')
                dow_crime = panel_df_clean.groupby(['year', 'day_of_week'])['crime_count'].mean().reset_index()

                try:
                    dow_pivot = dow_crime.pivot(index='day_of_week', columns='year', values='crime_count')

                    plt.figure(figsize=(10, 6))
                    sns.lineplot(data=dow_pivot)
                    plt.title('Day of Week Crime Patterns by Year')
                    plt.xlabel('Day of Week')
                    plt.ylabel('Average Crime Count')
                    plt.xticks(range(0, 7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
                    plt.legend(title='Year')
                    plt.grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.savefig(f"{viz_dir}/day_of_week_patterns.png", dpi=200, bbox_inches='tight')
                    plt.close()

                    log("Created day of week patterns visualization")
                except Exception as e:
                    log(f"Error creating day of week pivot: {str(e)}")
                    # Try alternative
                    try:
                        plt.figure(figsize=(10, 6))
                        for year in sorted(dow_crime['year'].unique()):
                            year_data = dow_crime[dow_crime['year'] == year]
                            plt.plot(year_data['day_of_week'], year_data['crime_count'], label=str(int(year)))
                        plt.title('Day of Week Crime Patterns by Year')
                        plt.xlabel('Day of Week')
                        plt.ylabel('Average Crime Count')
                        plt.xticks(range(0, 7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
                        plt.legend(title='Year')
                        plt.grid(True, alpha=0.3)
                        plt.tight_layout()
                        plt.savefig(f"{viz_dir}/day_of_week_patterns_alt.png", dpi=200, bbox_inches='tight')
                        plt.close()
                        log("Created alternative day of week visualization")
                    except Exception as e2:
                        log(f"Error creating alternative day of week visualization: {str(e2)}")
            except Exception as e:
                log(f"Error processing day of week data: {str(e)}")

    except Exception as e:
        log(f"Error creating seasonal patterns visualization: {str(e)}")

def add_fairness_mitigation(X_train, y_train, train_df, model_name="HistGradientBoostingRegressor"):
    """
    Apply fairness mitigation to training data with improved error handling

    Parameters:
    -----------
    X_train : DataFrame
        Training features
    y_train : Series
        Target variable
    train_df : DataFrame
        Complete training dataframe containing sensitive attributes
    model_name : str
        The model type to use (default: HistGradientBoostingRegressor)

    Returns:
    --------
    tuple
        Tuple containing (mitigated_X, mitigated_y, sample_weights)
    """
    if not FAIRLEARN_AVAILABLE:
        log("Fairlearn not available for fairness mitigation")
        return X_train, y_train, None

    try:
        log("Applying fairness mitigation techniques")

        # Detect sensitive attribute - try hardship_quartile first, then hardship_index if available
        sens_attr = None
        sens_attr_name = None

        if 'hardship_quartile' in train_df.columns:
            try:
                # Convert to string to avoid categorical issues
                sens_attr = train_df['hardship_quartile'].astype(str)
                sens_attr_name = 'hardship_quartile'
                log("Using 'hardship_quartile' as sensitive attribute")
            except Exception as e:
                log(f"Error converting hardship_quartile: {str(e)}")

        elif 'hardship_index' in train_df.columns:
            try:
                # Create quartiles if using continuous value
                train_df['hardship_index'] = pd.to_numeric(train_df['hardship_index'], errors='coerce')
                if train_df['hardship_index'].notna().any():
                    # Fill NaNs with median to allow quartile creation
                    train_df['hardship_index'] = train_df['hardship_index'].fillna(train_df['hardship_index'].median())

                    try:
                        # Try to create quartiles
                        quartiles = pd.qcut(train_df['hardship_index'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
                        sens_attr = quartiles.astype(str)
                        sens_attr_name = 'hardship_index_quartile'
                        log("Using 'hardship_index' (quartile-binned) as sensitive attribute")
                    except ValueError:  # Handle case with duplicate values
                        quartiles = pd.qcut(train_df['hardship_index'],
                                          [0, 0.25, 0.5, 0.75, 1],
                                          labels=['Q1', 'Q2', 'Q3', 'Q4'],
                                          duplicates='drop')
                        sens_attr = quartiles.astype(str)
                        sens_attr_name = 'hardship_index_quartile'
                        log("Using 'hardship_index' (quantile-binned) as sensitive attribute")
            except Exception as e:
                log(f"Error processing hardship_index: {str(e)}")

        if sens_attr is not None:
            mitigation_techniques = []

            # 1. Apply Reweighing (always available in fairlearn)
            try:
                reweighing = Reweighing()
                X_reweigh, y_reweigh, sample_weights_reweigh = reweighing.fit_transform(
                    X_train, y_train, sensitive_features=sens_attr
                )

                # Log information about weights
                weight_summary = pd.Series(sample_weights_reweigh).describe()
                log(f"Reweighing sample weights - min: {weight_summary['min']:.3f}, "
                    f"mean: {weight_summary['mean']:.3f}, max: {weight_summary['max']:.3f}")

                mitigation_techniques.append(('Reweighing', X_reweigh, y_reweigh, sample_weights_reweigh))
            except Exception as e:
                log(f"Error applying Reweighing: {str(e)}")

            # Evaluate each mitigation technique
            if mitigation_techniques:
                # Select the best technique (for now, default to Reweighing as it's most reliable)
                best_technique = mitigation_techniques[0]
                log(f"Using {best_technique[0]} as the fairness mitigation technique")

                # Create ethical warning based on disparity
                if sens_attr_name:
                    # Try to check for high disparity across protected groups
                    try:
                        # Group crime data by sensitive attribute
                        group_stats = train_df.groupby(sens_attr_name)['crime_count'].mean()
                        if group_stats.max() > 0 and group_stats.min() > 0:
                            max_rate = group_stats.max()
                            min_rate = group_stats.min()
                            disparity_ratio = max_rate / min_rate

                            if disparity_ratio > 3:
                                log("⚠️ ETHICAL WARNING - RED FLAG: High disparity detected across protected groups")
                                log(f"Crime rate ratio between highest and lowest groups: {disparity_ratio:.2f}x")
                                log("This may lead to biased predictions that reinforce existing inequalities.")
                            elif disparity_ratio > 1.5:
                                log("⚠️ ETHICAL WARNING - YELLOW FLAG: Moderate disparity detected")
                                log(f"Crime rate ratio between highest and lowest groups: {disparity_ratio:.2f}x")
                            else:
                                log("✓ Disparity across protected groups is within acceptable limits")
                    except Exception as e:
                        log(f"Error calculating disparity ratio: {str(e)}")

                return best_technique[1], best_technique[2], best_technique[3]
            else:
                log("No fairness mitigation techniques were successfully applied")
                return X_train, y_train, None
        else:
            log("No suitable sensitive attribute found for fairness mitigation")
            return X_train, y_train, None

    except Exception as e:
        log(f"Error applying fairness mitigation: {str(e)}")
        return X_train, y_train, None

# Add the main execution flow here, incorporating all the improved functions
# For example:

# ---------- 4. load datasets --------------------------------------
def main():
    # Load or fetch crime data
    crime_cache = f"{CACHE_DIR}/crime.parquet"
    comm_cache = f"{CACHE_DIR}/comm.parquet"
    census_cache = f"{CACHE_DIR}/census.parquet"
    bounds_cache = f"{CACHE_DIR}/bounds.parquet"

    if os.path.exists(crime_cache):
        crime = pd.read_parquet(crime_cache)
        log("Loading cached crime data")
    else:
        crime = fetch_batches(CRIME_API, BATCH_SIZE, MAX_RECORDS)
        if 'date' in crime.columns:
            crime['date'] = pd.to_datetime(crime['date'], errors='coerce')
        crime.to_parquet(crime_cache, index=False)
        log("Crime data downloaded and cached")

    if os.path.exists(comm_cache):
        comm = pd.read_parquet(comm_cache)
        log("Loading cached community data")
    else:
        comm = pd.DataFrame(requests.get(COMM_API, timeout=60).json())
        comm.to_parquet(comm_cache, index=False)
        log("Community data downloaded and cached")

    if os.path.exists(census_cache):
        census = pd.read_parquet(census_cache)
        log("Loading cached census data")
    else:
        try:
            census = pd.DataFrame(requests.get(CENSUS_API, timeout=60).json())
            census.to_parquet(census_cache, index=False)
            log("Census data downloaded and cached")
        except Exception as e:
            census = pd.DataFrame()
            log(f"Error fetching census data: {str(e)}")

    # Load or fetch boundaries for spatial analysis
    if os.path.exists(bounds_cache):
        bounds = pd.read_parquet(bounds_cache)
        log("Loading cached boundary data")
    else:
        try:
            bounds = pd.DataFrame(requests.get(BOUNDS_API, timeout=60).json())
            bounds.to_parquet(bounds_cache, index=False)
            log("Boundary data downloaded and cached")
        except Exception as e:
            bounds = pd.DataFrame()
            log(f"Error fetching boundary data: {str(e)}")

    # Apply year filter if specified
    if TIME_RANGE:
        original_count = len(crime)
        # Ensure date column is datetime and handle NaNs
        crime['date'] = pd.to_datetime(crime['date'], errors='coerce')
        valid_dates = ~crime['date'].isna()
        crime = crime[valid_dates & crime.date.dt.year.isin(TIME_RANGE)]
        log(f"Applied year filter {TIME_RANGE}: {len(crime):,} rows (from {original_count:,})")

    # Check data quality
    check_data_quality(crime, "Crime")
    check_data_quality(comm, "Community")
    if not census.empty:
        check_data_quality(census, "Census")

    # Clean any unhashable types in dataframes
    crime = clean_object_columns(crime)
    comm = clean_object_columns(comm)
    if not census.empty:
        census = clean_object_columns(census)
    if not bounds.empty:
        bounds = clean_object_columns(bounds)

    # Standardize community area columns
    crime, comm, census = standardize_community_area_columns(crime, comm, census)

    # Convert numeric columns in census data
    if not census.empty:
        numeric_cols = ['hardship_index', 'per_capita_income', 'percent_aged_16_unemployed',
                       'percent_aged_25_without_high_school_diploma', 'percent_households_below_poverty',
                       'percent_aged_under_18_or_over_64', 'percent_housing_crowded',
                       'percent_households_below_poverty', 'percent_minority']

        census = ensure_numeric(census, numeric_cols)

    # ---------- 5. enhanced feature engineering -----------------------
    log("Starting enhanced feature engineering")

    # Extract temporal features
    crime = extract_temporal_features(crime)

    # Create spatial features
    crime, comm = create_spatial_features(crime, comm)

    # Create crime type indicators with more detailed categories
    if 'primary_type' in crime.columns:
        # Violent crimes
        up = crime.primary_type.str.upper()
        crime['violent'] = up.isin(['HOMICIDE', 'ASSAULT', 'BATTERY', 'CRIMINAL SEXUAL ASSAULT', 'ROBBERY']).astype(int)

        # Public order crimes
        public_order_types = ['NARCOTICS', 'LIQUOR LAW VIOLATION', 'PUBLIC PEACE VIOLATION',
                             'INTERFERENCE WITH PUBLIC OFFICER', 'GAMBLING', 'OBSCENITY']
        crime['public_order'] = up.isin(public_order_types).astype(int)

        # Property crimes
        property_types = ['THEFT', 'BURGLARY', 'MOTOR VEHICLE THEFT', 'ARSON',
                         'CRIMINAL DAMAGE', 'CRIMINAL TRESPASS']
        crime['property'] = up.isin(property_types).astype(int)

        # Financial crimes
        financial_types = ['DECEPTIVE PRACTICE', 'FORGERY', 'FRAUD']
        crime['financial'] = up.isin(financial_types).astype(int)

        # Create severity levels based on FBI Uniform Crime Reporting
        part1_types = ['HOMICIDE', 'CRIMINAL SEXUAL ASSAULT', 'ROBBERY', 'ASSAULT',
                      'BURGLARY', 'THEFT', 'MOTOR VEHICLE THEFT', 'ARSON']
        crime['serious_crime'] = up.isin(part1_types).astype(int)

    # Create crime heatmap and visualizations
    # Use improved safe version
    create_crime_heatmap_safe(crime, comm, VIZ_DIR)

    # Get unique community areas
    community_areas = crime['community_area'].unique()

    # Create panel data structure (one row per community area per month)
    if USE_PANEL_DATA:
        panel_df = create_panel_data(crime, community_areas, TIME_RANGE)

        # Create seasonal patterns visualization
        plot_seasonal_patterns(panel_df, VIZ_DIR)

        # Join community data to panel
        comm['community_area'] = comm['community_area'].astype(str)
        panel_df['community_area'] = panel_df['community_area'].astype(str)

        panel_df = pd.merge(panel_df, comm, on='community_area', how='left')
        log(f"Merged panel data with community data: {panel_df.shape}")

        # Join census data if available
        if not census.empty:
            census['community_area'] = census['community_area'].astype(str)
            panel_df = pd.merge(panel_df, census, on='community_area', how='left')
            log(f"Merged panel data with census data: {panel_df.shape}")

        # Calculate crime rate if population data available
        TARGET = 'crime_count'  # Default target
        if USE_CRIME_RATE and 'population' in panel_df.columns:
            # Convert population to numeric if needed
            panel_df['population'] = pd.to_numeric(panel_df['population'], errors='coerce')
            log("Converted population to numeric")

            # Only use if we have valid values
            if 'population' in panel_df.columns and panel_df['population'].sum() > 0:
                # Replace zeros with 1 to avoid division by zero
                panel_df['population'] = panel_df['population'].replace(0, 1)
                panel_df['crime_rate'] = (panel_df.crime_count / (panel_df.population / 1000)).round(3)
                TARGET = 'crime_rate'
                log(f"Created {TARGET} as target variable")
            else:
                log(f"Using {TARGET} as target variable (population data invalid)")
        else:
            log(f"Using {TARGET} as target variable")

        # Create hardship quartiles for fairness analysis
        if 'hardship_index' in panel_df.columns:
            # Convert hardship_index to numeric if needed
            panel_df['hardship_index'] = pd.to_numeric(panel_df['hardship_index'], errors='coerce')
            log("Converted hardship_index to numeric")

            # Only create if we have valid values
            if not panel_df['hardship_index'].isna().all():
                # Fill missing values before creating quartiles
                panel_df['hardship_index'] = panel_df['hardship_index'].fillna(safe_median(panel_df['hardship_index']))
                try:
                    # Create quartiles - use string labels to avoid categorical dtype issues
                    panel_df['hardship_quartile'] = pd.qcut(
                        panel_df['hardship_index'],
                        q=4,
                        labels=['Low', 'Medium-Low', 'Medium-High', 'High']
                    ).astype(str)
                    log("Created hardship quartiles for fairness analysis")
                except ValueError:
                    # Handle case with duplicate values
                    try:
                        panel_df['hardship_quartile'] = pd.qcut(
                            panel_df['hardship_index'],
                            q=[0, 0.25, 0.5, 0.75, 1],
                            labels=['Low', 'Medium-Low', 'Medium-High', 'High'],
                            duplicates='drop'
                        ).astype(str)
                        log("Created hardship quartiles (with duplicate handling) for fairness analysis")
                    except Exception as e:
                        log(f"Could not create hardship quartiles: {str(e)}")
                        # Create a simple binary version as fallback
                        median_hardship = panel_df['hardship_index'].median()
                        panel_df['hardship_quartile'] = np.where(
                            panel_df['hardship_index'] > median_hardship,
                            'High',
                            'Low'
                        )
                        log("Created binary hardship indicator as fallback")

        # Make sure the dataframe has no unhashable types
        panel_df = clean_object_columns(panel_df)

        # Filter out highly correlated features - use safer version
        columns_to_keep = filter_highly_correlated_features_safe(
            panel_df,
            TARGET,
            threshold=MAX_CORRELATION
        )

        panel_df = panel_df[columns_to_keep]
        log(f"Panel data shape after correlation filtering: {panel_df.shape}")

        # Convert month_year to string for proper data splitting
        if 'month_year' in panel_df.columns:
            panel_df['month_year_str'] = panel_df['month_year'].astype(str)

        # Split data by time
        if len(TIME_RANGE) >= 3:
            train_mask = panel_df['year'] == TIME_RANGE[0]  # First year for training
            val_mask = panel_df['year'] == TIME_RANGE[1]    # Second year for validation
            test_mask = panel_df['year'] == TIME_RANGE[-1]  # Last year for testing

            train_df = panel_df[train_mask].copy()
            val_df = panel_df[val_mask].copy()
            test_df = panel_df[test_mask].copy()

            log(f"Train data: {train_df.shape} rows from {TIME_RANGE[0]}")
            log(f"Validation data: {val_df.shape} rows from {TIME_RANGE[1]}")
            log(f"Test data: {test_df.shape} rows from {TIME_RANGE[-1]}")
        else:
            # Fallback if not enough years in TIME_RANGE
            log("Not enough years in TIME_RANGE for proper temporal split. Using random split.")
            train_val_df, test_df = train_test_split(panel_df, test_size=0.2, random_state=RAND_STATE)
            train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=RAND_STATE)

            log(f"Train data: {train_df.shape} rows (random split)")
            log(f"Validation data: {val_df.shape} rows (random split)")
            log(f"Test data: {test_df.shape} rows (random split)")

        # Prepare features and target
        # Drop non-numeric and administrative columns
        exclude_cols = [
            'month_year', 'month_year_str', TARGET,
            'community_area', 'geometry', 'the_geom',
            'community', 'area_numbe', 'shape_area', 'shape_len'
        ]

        feature_cols = [col for col in panel_df.columns if col not in exclude_cols]

        # Select only numeric columns
        X_train = train_df[feature_cols].select_dtypes(include=['number'])
        y_train = train_df[TARGET]

        X_val = val_df[feature_cols].select_dtypes(include=['number'])
        y_val = val_df[TARGET]

        X_test = test_df[feature_cols].select_dtypes(include=['number'])
        y_test = test_df[TARGET]

        # Feature selection
        selected_features = select_features(X_train, y_train, MAX_FEATURES)

        X_train = X_train[selected_features]
        X_val = X_val[selected_features]
        X_test = X_test[selected_features]

        log(f"Final feature set: {X_train.shape[1]} features")

        # Impute missing values before modeling
        X_train = impute_missing_values(X_train)
        X_val = impute_missing_values(X_val)
        X_test = impute_missing_values(X_test)

        log(f"Data shapes after imputation - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

        # Log feature correlations with target
        if len(selected_features) > 0:
            # Calculate correlations safely
            corr_with_target = pd.Series(index=X_train.columns, dtype=float)
            for col in X_train.columns:
                try:
                    # Use only non-NA values in both columns
                    mask = ~X_train[col].isna() & ~y_train.isna()
                    if mask.sum() > 10:  # Only calculate if we have enough valid points
                        corr = np.corrcoef(X_train[col][mask], y_train[mask])[0, 1]
                        corr_with_target[col] = abs(corr)
                except Exception as e:
                    log(f"Error calculating correlation for {col}: {str(e)}")
                    corr_with_target[col] = 0.0

            corr_with_target = corr_with_target.sort_values(ascending=False)
            log("Top feature correlations with target:")
            for i, (col, corr) in enumerate(corr_with_target.items()):
                if i < 10:  # Show top 10
                    log(f"  {i+1}. {col}: {corr:.3f}")

        # Extract protected attributes for fairness analysis
        protected_attrs = {}

        if 'hardship_quartile' in test_df.columns:
            protected_attrs['hardship_quartile'] = test_df['hardship_quartile']

        if 'hardship_index' in test_df.columns:
            protected_attrs['hardship_index'] = test_df['hardship_index']

        if protected_attrs:
            log(f"Protected attributes for fairness analysis: {list(protected_attrs.keys())}")

        # Standardize features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)

        # Convert to DataFrames to keep column names
        X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
        X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
        X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

        # Check for any remaining NaNs after scaling
        if X_train_scaled_df.isna().any().any():
            log(f"WARNING: {X_train_scaled_df.isna().sum().sum()} NaNs remain in training data after scaling")
            X_train_scaled_df = X_train_scaled_df.fillna(0)  # Fill any remaining NaNs
        if X_val_scaled_df.isna().any().any():
            log(f"WARNING: {X_val_scaled_df.isna().sum().sum()} NaNs remain in validation data after scaling")
            X_val_scaled_df = X_val_scaled_df.fillna(0)
        if X_test_scaled_df.isna().any().any():
            log(f"WARNING: {X_test_scaled_df.isna().sum().sum()} NaNs remain in test data after scaling")
            X_test_scaled_df = X_test_scaled_df.fillna(0)

        # ---------- 6. model training and evaluation -------------------
        log("Training models with time-based validation")

        # Dictionary to store model results
        model_results = {}
        trained_models = {}

        # Apply fairness mitigation if enabled
        if APPLY_FAIRNESS and FAIRLEARN_AVAILABLE:
            # Apply fairness mitigation before model training
            X_train_mit, y_train_mit, sample_weights = add_fairness_mitigation(
                X_train_scaled_df, y_train, train_df
            )
        else:
            X_train_mit, y_train_mit, sample_weights = X_train_scaled_df, y_train, None

        # 1. Ridge Regression using Pipeline with imputation
        log("Training Ridge Regression model")

        best_alpha = 1.0  # Default value

        try:
            # Define objective function for Ridge alpha tuning with built-in imputation
            if OPTUNA_AVAILABLE:
                def objective_ridge(trial):
                    alpha = trial.suggest_float("alpha", 0.01, 100.0, log=True)

                    # Create a pipeline with imputation
                    pipeline = Pipeline([
                        ('imputer', SimpleImputer(strategy='median')),
                        ('ridge', Ridge(alpha=alpha, random_state=RAND_STATE))
                    ])

                    # Fit and evaluate
                    if sample_weights is not None:
                        pipeline.fit(X_train_mit, y_train_mit, ridge__sample_weight=sample_weights)
                    else:
                        pipeline.fit(X_train_mit, y_train_mit)

                    # Calculate RMSE
                    return np.sqrt(mean_squared_error(y_val, pipeline.predict(X_val_scaled_df)))

                # Run optimization with error handling
                try:
                    study = optuna.create_study(direction="minimize")
                    study.optimize(objective_ridge, n_trials=N_TRIALS)
                    best_alpha = study.best_params["alpha"]
                    log(f"Best Ridge alpha: {best_alpha:.4f}")
                except Exception as e:
                    log(f"Error during Ridge hyperparameter optimization: {str(e)}")
                    best_alpha = 1.0  # Fallback to default value

            # Create pipeline with imputation
            ridge_pipeline = Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('ridge', Ridge(alpha=best_alpha, random_state=RAND_STATE))
            ])

            # Train the pipeline
            if sample_weights is not None:
                ridge_pipeline.fit(X_train_mit, y_train_mit, ridge__sample_weight=sample_weights)
            else:
                ridge_pipeline.fit(X_train_mit, y_train_mit)

            # Evaluate on test set
            ridge_pred = ridge_pipeline.predict(X_test_scaled_df)
            ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
            ridge_r2 = r2_score(y_test, ridge_pred)
            ridge_mae = mean_absolute_error(y_test, ridge_pred)

            log(f"Ridge - RMSE: {ridge_rmse:.3f}, R²: {ridge_r2:.3f}, MAE: {ridge_mae:.3f}")
            model_results['Ridge'] = {'rmse': ridge_rmse, 'r2': ridge_r2, 'mae': ridge_mae}
            trained_models['Ridge'] = (ridge_pipeline, ridge_pred)

            # Log important coefficients
            ridge_model = ridge_pipeline.named_steps['ridge']
            coef = pd.Series(ridge_model.coef_, index=X_train.columns)
            important_coef = coef.abs().sort_values(ascending=False)
            log("Top Ridge coefficients:")
            for i, (col, val) in enumerate(important_coef.items()):
                if i < 5:  # Show top 5
                    log(f"  {i+1}. {col}: {val:.4f}")

        except Exception as e:
            log(f"Error training Ridge model: {str(e)}")

        # 2. Gradient Boosting using HistGradientBoostingRegressor (handles missing values)
        log("Training Gradient Boosting model")

        # Default parameters
        best_params = {
            'max_iter': 100,
            'max_depth': 3,
            'learning_rate': 0.1,
            'max_bins': 255,
            'early_stopping': True,
            'random_state': RAND_STATE
        }

        try:
            if OPTUNA_AVAILABLE:
                # Define objective function for GB
                def objective_gb(trial):
                    params = {
                        'max_iter': trial.suggest_int("max_iter", 50, 300),
                        'max_depth': trial.suggest_int("max_depth", 2, 6),
                        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
                        'max_bins': trial.suggest_int("max_bins", 32, 255),
                        'early_stopping': True,
                        'random_state': RAND_STATE
                    }

                    # Use HistGradientBoostingRegressor which handles missing values natively
                    model = HistGradientBoostingRegressor(**params)

                    if sample_weights is not None:
                        model.fit(X_train_mit, y_train_mit, sample_weight=sample_weights)
                    else:
                        model.fit(X_train_mit, y_train_mit)

                    # Calculate RMSE
                    return np.sqrt(mean_squared_error(y_val, model.predict(X_val_scaled_df)))

                # Run optimization with error handling
                try:
                    study = optuna.create_study(direction="minimize")
                    study.optimize(objective_gb, n_trials=N_TRIALS)
                    best_params = study.best_params
                    best_params['random_state'] = RAND_STATE
                    best_params['early_stopping'] = True
                    log(f"Best GB parameters: {best_params}")
                except Exception as e:
                    log(f"Error during GB hyperparameter optimization: {str(e)}")
                    # Use default parameters

            # Train with best/default parameters using HistGradientBoostingRegressor
            gb = HistGradientBoostingRegressor(**best_params)

            if sample_weights is not None:
                gb.fit(X_train_mit, y_train_mit, sample_weight=sample_weights)
            else:
                gb.fit(X_train_mit, y_train_mit)

            # Evaluate on test set
            gb_pred = gb.predict(X_test_scaled_df)
            gb_rmse = np.sqrt(mean_squared_error(y_test, gb_pred))
            gb_r2 = r2_score(y_test, gb_pred)
            gb_mae = mean_absolute_error(y_test, gb_pred)

            log(f"Gradient Boosting - RMSE: {gb_rmse:.3f}, R²: {gb_r2:.3f}, MAE: {gb_mae:.3f}")
            model_results['Gradient Boosting'] = {'rmse': gb_rmse, 'r2': gb_r2, 'mae': gb_mae}
            trained_models['Gradient Boosting'] = (gb, gb_pred)

            # Extract feature importances with better fallback options
            try:
                if hasattr(gb, 'feature_importances_'):
                    feature_importances = gb.feature_importances_
                elif hasattr(gb, '_estimator') and hasattr(gb._estimator, 'feature_importances_'):
                    feature_importances = gb._estimator.feature_importances_
                else:
                    # Create a surrogate model for feature importance
                    log("Creating surrogate GradientBoostingRegressor for feature importance")
                    surrogate_gb = GradientBoostingRegressor(n_estimators=100, random_state=RAND_STATE)
                    surrogate_gb.fit(X_train_scaled_df, y_train)
                    feature_importances = surrogate_gb.feature_importances_

                # Report feature importances
                importances = pd.Series(feature_importances, index=X_train.columns)
                importances = importances.sort_values(ascending=False)

                log("Top GB feature importances:")
                for i, (col, val) in enumerate(importances.items()):
                    if i < 5:  # Show top 5
                        log(f"  {i+1}. {col}: {val:.4f}")
            except Exception as e:
                log(f"Error extracting feature importances: {str(e)}")

        except Exception as e:
            log(f"Error training Gradient Boosting model: {str(e)}")

        # 3. Mixed Effects Model if available
        if MIXED_MODELS_AVAILABLE:
            log("Training Mixed Effects Model")

            try:
                # Get top features from correlations with target
                top_indices = corr_with_target.index[:min(3, len(corr_with_target))]

                # Create dataframe for mixed effects model
                me_df = pd.DataFrame()
                me_df['y'] = y_train
                for feat in top_indices:
                    me_df[feat] = X_train[feat]

                # Add community area as grouping variable
                me_df['community_area'] = train_df['community_area'].values

                # Impute any missing values
                for col in me_df.columns:
                    if col != 'community_area' and pd.api.types.is_numeric_dtype(me_df[col]) and me_df[col].isna().any():
                        me_df[col] = me_df[col].fillna(safe_median(me_df[col]))

                # Fit mixed effects model
                formula = f"y ~ {' + '.join(top_indices)}"
                mixed_model = smf.mixedlm(
                    formula,
                    me_df,
                    groups=me_df['community_area']
                )

                # Handle convergence warnings
                try:
                    mixed_result = mixed_model.fit()
                except:
                    # Try with different optimizer
                    log("Mixed model optimization failed. Trying with alternative optimizer.")
                    mixed_result = mixed_model.fit(method='powell')

                # Create test dataframe for prediction
                me_test = pd.DataFrame()
                for feat in top_indices:
                    me_test[feat] = X_test[feat]

                # Impute missing values in test data
                for col in me_test.columns:
                    if pd.api.types.is_numeric_dtype(me_test[col]) and me_test[col].isna().any():
                        me_test[col] = me_test[col].fillna(safe_median(me_test[col]))

                # Predict with fixed effects only (simplification)
                mixed_pred = mixed_result.predict(me_test)

                # Evaluate
                mixed_rmse = np.sqrt(mean_squared_error(y_test, mixed_pred))
                mixed_r2 = r2_score(y_test, mixed_pred)
                mixed_mae = mean_absolute_error(y_test, mixed_pred)

                log(f"Mixed Effects Model - RMSE: {mixed_rmse:.3f}, R²: {mixed_r2:.3f}, MAE: {mixed_mae:.3f}")
                model_results['Mixed Effects'] = {'rmse': mixed_rmse, 'r2': mixed_r2, 'mae': mixed_mae}
                trained_models['Mixed Effects'] = (mixed_result, mixed_pred)

                # Log model summary
                log("Mixed Effects Model Summary:")
                for param, value in mixed_result.params.items():
                    log(f"  {param}: {value:.4f}")

                # Add random effects groups variance
                if hasattr(mixed_result, 'cov_re'):
                    cov_re_value = mixed_result.cov_re
                    # Handle multi-dimensional cov_re
                    if hasattr(cov_re_value, 'item'):
                        log(f"  Group Var: {cov_re_value.item():.4f}")
                    else:
                        log(f"  Group Var: {safe_convert(cov_re_value):.4f}")

            except Exception as e:
                log(f"Error training Mixed Effects model: {str(e)}")

        # 4. Baseline model for comparison
        log("Training baseline model")
        try:
            # Create a pipeline with proper imputation
            baseline_pipeline = Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('model', DummyRegressor(strategy='mean'))
            ])

            # Train
            baseline_pipeline.fit(X_train, y_train)
            baseline_pred = baseline_pipeline.predict(X_test)

            baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))
            baseline_r2 = r2_score(y_test, baseline_pred)
            baseline_mae = mean_absolute_error(y_test, baseline_pred)

            log(f"Baseline (Mean) - RMSE: {baseline_rmse:.3f}, R²: {baseline_r2:.3f}, MAE: {baseline_mae:.3f}")
            model_results['Baseline'] = {'rmse': baseline_rmse, 'r2': baseline_r2, 'mae': baseline_mae}
            trained_models['Baseline'] = (baseline_pipeline, baseline_pred)
        except Exception as e:
            log(f"Error training Baseline model: {str(e)}")

        # ---------- 7. model evaluation and fairness analysis ------------
        log("Performing model evaluation and fairness analysis")

        # Identify best model
        best_model_name = None
        best_rmse = float('inf')
        for model_name, metrics in model_results.items():
            if 'rmse' in metrics and metrics['rmse'] < best_rmse:
                best_rmse = metrics['rmse']
                best_model_name = model_name

        if best_model_name:
            log(f"Best model based on RMSE: {best_model_name} ({best_rmse:.3f})")
        else:
            log("No valid models found for comparison")

        # Plot model comparison
        try:
            plot_model_comparison(model_results, VIZ_DIR)
            log("Created model comparison visualization")
        except Exception as e:
            log(f"Error creating model comparison plot: {str(e)}")

        # Feature importance visualization for GB model
        if 'Gradient Boosting' in trained_models:
            try:
                gb_model, _ = trained_models['Gradient Boosting']

                # Create feature importance visualization
                try:
                    # Extract feature importances with proper error handling
                    feature_importances = None
                    if hasattr(gb_model, 'feature_importances_'):
                        feature_importances = gb_model.feature_importances_
                    elif hasattr(gb_model, '_estimator') and hasattr(gb_model._estimator, 'feature_importances_'):
                        feature_importances = gb_model._estimator.feature_importances_
                    else:
                        # Create a surrogate model for feature importance
                        log("Creating surrogate GradientBoostingRegressor for feature importance")
                        surrogate_gb = GradientBoostingRegressor(n_estimators=100, random_state=RAND_STATE)
                        surrogate_gb.fit(X_train_scaled_df, y_train)
                        feature_importances = surrogate_gb.feature_importances_

                    plt.figure(figsize=(12, 8))
                    plt.title('Feature Importances (Gradient Boosting)')
                    importances = pd.Series(feature_importances, index=X_train.columns)
                    importances = importances.sort_values(ascending=False)
                    importances.head(20).plot(kind='barh')
                    plt.xlabel('Relative Importance')
                    plt.tight_layout()
                    plt.savefig(f"{VIZ_DIR}/feature_importance_gb.png", dpi=200, bbox_inches='tight')
                    plt.close()

                    log("Created feature importance visualization")
                except Exception as e:
                    log(f"Error creating feature importance visualization: {str(e)}")

                # Create SHAP visualizations for explainability
                if USE_EXPLAINERS and SHAP_AVAILABLE:
                    try:
                        shap_values = create_shap_visualizations(
                            gb_model,
                            X_test_scaled_df,
                            X_test.columns,
                            VIZ_DIR
                        )
                        log("Created SHAP visualizations")
                    except Exception as e:
                        log(f"Error creating SHAP visualizations: {str(e)}")

                # Create LIME explanations
                if USE_EXPLAINERS and LIME_AVAILABLE:
                    try:
                        create_lime_explanations(
                            gb_model,
                            X_train_scaled_df,
                            X_test_scaled_df,
                            X_test.columns,
                            VIZ_DIR
                        )
                        log("Created LIME explanations")
                    except Exception as e:
                        log(f"Error creating LIME explanations: {str(e)}")

            except Exception as e:
                log(f"Error in model explainability: {str(e)}")

        # Ridge coefficient visualization
        if 'Ridge' in trained_models:
            try:
                ridge_pipeline, _ = trained_models['Ridge']

                # Get the Ridge model from the pipeline
                ridge_model = ridge_pipeline.named_steps.get('ridge')

                if ridge_model is not None and hasattr(ridge_model, 'coef_'):
                    plt.figure(figsize=(12, 8))
                    plt.title('Feature Coefficients (Ridge)')
                    coef = pd.Series(ridge_model.coef_, index=X_train.columns)
                    coef = coef.reindex(coef.abs().sort_values(ascending=False).index)
                    coef.head(20).plot(kind='barh')
                    plt.xlabel('Coefficient Value')
                    plt.tight_layout()
                    plt.savefig(f"{VIZ_DIR}/coefficients_ridge.png", dpi=200, bbox_inches='tight')
                    plt.close()
                    log("Created Ridge coefficient visualization")
                else:
                    log("Could not access Ridge coefficients")
            except Exception as e:
                log(f"Error creating Ridge coefficient visualization: {str(e)}")

        # Actual vs Predicted plot for best model
        if best_model_name in trained_models:
            _, best_pred = trained_models[best_model_name]

            try:
                plt.figure(figsize=(10, 8))
                plt.scatter(y_test, best_pred, alpha=0.6)
                plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
                plt.xlabel('Actual')
                plt.ylabel('Predicted')
                plt.title(f'Actual vs Predicted ({best_model_name})')
                plt.tight_layout()
                plt.savefig(f"{VIZ_DIR}/actual_vs_predicted.png", dpi=200, bbox_inches='tight')
                plt.close()
                log("Created actual vs predicted plot")

                # Residual plot
                residuals = y_test - best_pred
                plt.figure(figsize=(10, 8))
                plt.scatter(best_pred, residuals, alpha=0.6)
                plt.hlines(y=0, xmin=best_pred.min(), xmax=best_pred.max(), colors='k', linestyles='--')
                plt.xlabel('Predicted')
                plt.ylabel('Residuals')
                plt.title(f'Residual Plot ({best_model_name})')
                plt.tight_layout()
                plt.savefig(f"{VIZ_DIR}/residual_plot.png", dpi=200, bbox_inches='tight')
                plt.close()
                log("Created residual plot")

                # Create a heatmap visualization of predictions by community area
                if FOLIUM_AVAILABLE and 'community_area' in test_df.columns:
                    try:
                        # Get community area predictions
                        community_predictions = pd.DataFrame({
                            'community_area': test_df['community_area'],
                            'actual': y_test,
                            'predicted': best_pred,
                            'error': np.abs(y_test - best_pred)
                        })

                        # Aggregate by community area
                        community_results = community_predictions.groupby('community_area').agg({
                            'actual': 'mean',
                            'predicted': 'mean',
                            'error': 'mean'
                        }).reset_index()

                        try:
                            # Check if coordinate columns exist in the community data
                            if 'center_lat' not in comm.columns or 'center_lon' not in comm.columns:
                                log("Missing coordinate columns in community data. Creating them from crime data.")

                                # Create centroid coordinates from crime data
                                crime_coords = crime.dropna(subset=['latitude', 'longitude'])

                                # Group by community area and calculate mean coordinates
                                centroids = crime_coords.groupby('community_area').agg({
                                    'latitude': 'mean',
                                    'longitude': 'mean'
                                }).reset_index()

                                # Rename columns to match expected names
                                centroids.rename(columns={
                                    'latitude': 'center_lat',
                                    'longitude': 'center_lon'
                                }, inplace=True)

                                # Merge with community data
                                comm = pd.merge(
                                    comm,
                                    centroids,
                                    on='community_area',
                                    how='left'
                                )

                                log(f"Created centroid coordinates for {centroids.shape[0]} community areas")

                            # Merge with community data for coordinates
                            prediction_map_data = pd.merge(
                                community_results,
                                comm[['community_area', 'center_lat', 'center_lon']],
                                on='community_area',
                                how='left'
                            )

                            # Convert coordinates to numeric
                            prediction_map_data['center_lat'] = pd.to_numeric(prediction_map_data['center_lat'], errors='coerce')
                            prediction_map_data['center_lon'] = pd.to_numeric(prediction_map_data['center_lon'], errors='coerce')

                            # Filter out NaN coordinates
                            prediction_map_data = prediction_map_data.dropna(subset=['center_lat', 'center_lon'])

                            # Create map with predictions
                            m_pred = folium.Map(location=[41.8781, -87.6298], zoom_start=10,
                                            tiles='CartoDB positron')

                            # Add circles for predicted values
                            for _, row in prediction_map_data.iterrows():
                                try:
                                    ca_lat = float(row['center_lat'])
                                    ca_lon = float(row['center_lon'])
                                    predicted_val = float(row['predicted'])
                                    actual_val = float(row['actual'])
                                    error_val = float(row['error'])

                                    # Scale prediction for circle size (protect against negative values)
                                    circle_radius = np.log1p(max(predicted_val, 0.1)) * 2

                                    # Add circle for predicted value
                                    folium.CircleMarker(
                                        location=[ca_lat, ca_lon],
                                        radius=circle_radius,
                                        color='blue',
                                        fill=True,
                                        fill_color='blue',
                                        fill_opacity=0.5,
                                        popup=f"Community Area: {row['community_area']}<br>"
                                            f"Predicted: {predicted_val:.1f}<br>"
                                            f"Actual: {actual_val:.1f}<br>"
                                            f"Error: {error_val:.1f}"
                                    ).add_to(m_pred)
                                except (ValueError, TypeError) as e:
                                    # Skip problematic rows
                                    continue

                            # Save map
                            pred_map_path = f"{VIZ_DIR}/predicted_crime_map.html"
                            m_pred.save(pred_map_path)
                            log(f"Saved prediction map to {pred_map_path}")

                            # Create map of prediction errors
                            m_error = folium.Map(location=[41.8781, -87.6298], zoom_start=10,
                                            tiles='CartoDB positron')

                            # Find max error for scaling
                            max_error = prediction_map_data['error'].max()

                            # Add circles for prediction error
                            for _, row in prediction_map_data.iterrows():
                                try:
                                    ca_lat = float(row['center_lat'])
                                    ca_lon = float(row['center_lon'])
                                    predicted_val = float(row['predicted'])
                                    actual_val = float(row['actual'])
                                    error_val = float(row['error'])

                                    # Scale error for circle size
                                    circle_radius = (error_val / max_error) * 15 if max_error > 0 else 5

                                    # Add circle with error
                                    folium.CircleMarker(
                                        location=[ca_lat, ca_lon],
                                        radius=circle_radius,
                                        color='red',
                                        fill=True,
                                        fill_color='red',
                                        fill_opacity=0.5,
                                        popup=f"Community Area: {row['community_area']}<br>"
                                            f"Error: {error_val:.1f}<br>"
                                            f"Actual: {actual_val:.1f}<br>"
                                            f"Predicted: {predicted_val:.1f}"
                                    ).add_to(m_error)
                                except (ValueError, TypeError) as e:
                                    # Skip problematic rows
                                    continue

                            # Save error map
                            error_map_path = f"{VIZ_DIR}/prediction_error_map.html"
                            m_error.save(error_map_path)
                            log(f"Saved error map to {error_map_path}")
                        except Exception as e:
                            log(f"Error creating prediction maps: {str(e)}")
                            import traceback
                            log(f"Detailed error: {traceback.format_exc()}")
                    except Exception as e:
                        log(f"Error creating prediction maps: {str(e)}")
            except Exception as e:
                log(f"Error creating prediction plots: {str(e)}")

        # Fairness analysis
        fairness_results = {}
        if protected_attrs and best_model_name in trained_models:
            try:
                _, best_pred = trained_models[best_model_name]
                fairness_results = perform_fairness_analysis(y_test, best_pred, protected_attrs, VIZ_DIR)
                log("Completed fairness analysis")
            except Exception as e:
                log(f"Error performing fairness analysis: {str(e)}")

        # ---------- 8. save results and models -------------------------------
        log("Saving results and model information")

        # Save model results
        try:
            results_data = {
                'metadata': {
                    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'target_variable': TARGET,
                    'years': TIME_RANGE,
                    'data_shape': panel_df.shape,
                    'train_shape': X_train.shape,
                    'val_shape': X_val.shape,
                    'test_shape': X_test.shape,
                    'best_model': best_model_name,
                    'features_used': X_train.columns.tolist(),
                    'fairness_mitigation_applied': APPLY_FAIRNESS and FAIRLEARN_AVAILABLE
                },
                'model_results': {
                    model: {
                        metric: convert_to_serializable(value)
                        for metric, value in metrics.items()
                    }
                    for model, metrics in model_results.items()
                },
                'fairness_results': convert_to_serializable(fairness_results)
            }

            # Save results to JSON
            results_path = f"{MODELS_DIR}/results_{datetime.now():%Y%m%d_%H%M%S}.json"
            with open(results_path, 'w') as f:
                json.dump(results_data, f, indent=2)
            log(f"Saved results to {results_path}")

            # Save best model using MLflow if available
            if USE_MLFLOW and MLFLOW_AVAILABLE and best_model_name in trained_models:
                with mlflow.start_run(run_name=f"predictive_policing_{best_model_name}"):
                    # Log parameters
                    mlflow.log_param("target_variable", TARGET)
                    mlflow.log_param("time_range", TIME_RANGE)
                    mlflow.log_param("feature_count", X_train.shape[1])
                    mlflow.log_param("fairness_mitigation", APPLY_FAIRNESS and FAIRLEARN_AVAILABLE)

                    # Log metrics
                    for metric, value in model_results[best_model_name].items():
                        mlflow.log_metric(metric, value)

                    # Log feature importances if available for Gradient Boosting
                    if best_model_name == 'Gradient Boosting':
                        gb_model, _ = trained_models[best_model_name]
                        feature_importances = None

                        # Extract feature importances
                        if hasattr(gb_model, 'feature_importances_'):
                            feature_importances = pd.DataFrame({
                                'feature': X_train.columns,
                                'importance': gb_model.feature_importances_
                            }).sort_values('importance', ascending=False)
                        elif hasattr(gb_model, '_estimator') and hasattr(gb_model._estimator, 'feature_importances_'):
                            feature_importances = pd.DataFrame({
                                'feature': X_train.columns,
                                'importance': gb_model._estimator.feature_importances_
                            }).sort_values('importance', ascending=False)
                        else:
                            # Create a surrogate model
                            surrogate_gb = GradientBoostingRegressor(n_estimators=100, random_state=RAND_STATE)
                            surrogate_gb.fit(X_train_scaled_df, y_train)
                            feature_importances = pd.DataFrame({
                                'feature': X_train.columns,
                                'importance': surrogate_gb.feature_importances_
                            }).sort_values('importance', ascending=False)

                        if feature_importances is not None:
                            # Log as JSON artifact
                            feature_imp_path = f"{MODELS_DIR}/feature_importance_{datetime.now():%Y%m%d_%H%M%S}.json"
                            feature_importances.to_json(feature_imp_path, orient='records')
                            mlflow.log_artifact(feature_imp_path)

                    # Log model
                    if best_model_name == 'Ridge':
                        mlflow.sklearn.log_model(trained_models[best_model_name][0], "ridge_model")
                    elif best_model_name == 'Gradient Boosting':
                        mlflow.sklearn.log_model(trained_models[best_model_name][0], "gb_model")

                    # Log environment info
                    conda_env = {
                        'channels': ['defaults', 'conda-forge'],
                        'dependencies': [
                            'python=3.9',
                            'pandas',
                            'numpy',
                            'scikit-learn',
                            'matplotlib',
                            'seaborn',
                            'mlflow',
                            'folium',
                            'shap',
                            'lime',
                            'fairlearn',
                            'statsmodels'
                        ],
                        'name': 'predictive_policing_env'
                    }

                    # Save conda env
                    conda_path = f"{MODELS_DIR}/conda_env_{datetime.now():%Y%m%d_%H%M%S}.yaml"
                    with open(conda_path, 'w') as f:
                        yaml.dump(conda_env, f)
                    mlflow.log_artifact(conda_path)

                    # Log fairness report if available
                    if fairness_results:
                        fairness_path = f"{MODELS_DIR}/fairness_report_{datetime.now():%Y%m%d_%H%M%S}.json"
                        with open(fairness_path, 'w') as f:
                            json.dump(convert_to_serializable(fairness_results), f, indent=2)
                        mlflow.log_artifact(fairness_path)

                    log(f"Saved model artifacts to MLflow")
            else:
                # Traditional model saving
                import pickle
                if best_model_name in trained_models:
                    model_path = f"{MODELS_DIR}/{best_model_name.lower().replace(' ', '_')}_{datetime.now():%Y%m%d_%H%M%S}.pkl"
                    with open(model_path, 'wb') as f:
                        pickle.dump(trained_models[best_model_name][0], f)
                    log(f"Saved {best_model_name} model to {model_path}")
                else:
                    log("No best model to save")

        except Exception as e:
            log(f"Error saving results: {str(e)}")

        # ---------- 9. deployment preparation -----------------------------
        log("Preparing for model deployment")

        # Export preprocessing steps and model for deployment
        try:
            # Create a simplified pipeline for deployment with MLflow if available
            if USE_MLFLOW and MLFLOW_AVAILABLE and best_model_name in trained_models:
                # Create a preprocessing function and model wrapper class
                preprocessing_code = f"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from mlflow.pyfunc import PythonModel

class CrimePredictionModel(PythonModel):
    \"\"\"
    MLflow wrapper for crime prediction model
    \"\"\"
    def __init__(self):
        \"\"\"Initialize the model wrapper\"\"\"
        self.model = None
        self.scaler = StandardScaler()
        self.scaler.mean_ = {scaler.mean_.tolist() if hasattr(scaler, 'mean_') else None}
        self.scaler.scale_ = {scaler.scale_.tolist() if hasattr(scaler, 'scale_') else None}
        self.features = {X_train.columns.tolist()}

    def load_context(self, context):
        \"\"\"Load the model from the MLflow artifact\"\"\"
        import pickle
        model_path = context.artifacts["model"]
        with open(model_path, "rb") as f:
            self.model = pickle.load(f)

    def preprocess(self, data):
        \"\"\"Preprocess the data for prediction\"\"\"
        # Ensure all required features are present
        missing_features = [f for f in self.features if f not in data.columns]
        if missing_features:
            for feature in missing_features:
                data[feature] = 0  # Default value for missing features

        # Select and order features
        X = data[self.features]

        # Fill missing values
        for col in X.columns:
            if X[col].isna().any():
                X[col] = X[col].fillna(X[col].median() if X[col].dtype != 'object' else 'UNKNOWN')

        # Scale features
        X_scaled = pd.DataFrame(
            self.scaler.transform(X),
            columns=self.features,
            index=X.index
        )

        return X_scaled

    def predict(self, context, data):
        \"\"\"Make predictions using the model\"\"\"
        # Convert to DataFrame if necessary
        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame(data)

        # Preprocess the data
        X_processed = self.preprocess(data)

        # Make predictions
        predictions = self.model.predict(X_processed)

        return predictions
"""

                # Save preprocessing code
                model_wrapper_path = f"{MODELS_DIR}/model_wrapper_{datetime.now():%Y%m%d_%H%M%S}.py"
                with open(model_wrapper_path, 'w') as f:
                    f.write(preprocessing_code)
                log(f"Saved MLflow model wrapper to {model_wrapper_path}")

                # Create a conda.yaml file for the model
                conda_env = {
                    'channels': ['defaults', 'conda-forge'],
                    'dependencies': [
                        'python=3.9',
                        'pandas',
                        'numpy',
                        'scikit-learn',
                        'matplotlib',
                        'seaborn',
                        'mlflow',
                        'shap',
                        'lime',
                        'fairlearn'
                    ],
                    'name': 'crime_prediction_env'
                }

                conda_path = f"{MODELS_DIR}/conda_env_{datetime.now():%Y%m%d_%H%M%S}.yaml"
                with open(conda_path, 'w') as f:
                    yaml.dump(conda_env, f)
                log(f"Created conda environment file for deployment")

            else:
                # Create a traditional preprocessing script
                preprocessing_code = f"""
# Preprocessing code for deployment
import pandas as pd
import numpy as np

def safe_median(s):
    \"\"\"Safely compute median of a Series, handling NaN values and conversion issues\"\"\"
    try:
        median = s.dropna().median()
        if hasattr(median, 'item'):
            return float(median.item())
        else:
            return float(median) if pd.notna(median) else 0.0
    except (TypeError, ValueError):
        return 0.0

def preprocess_for_prediction(raw_data):
    \"\"\"
    Preprocess raw data for prediction with trained model.

    Parameters:
    -----------
    raw_data : dict or DataFrame
        Raw input data with community area and time features

    Returns:
    --------
    processed_features : DataFrame
        Processed features ready for model prediction
    \"\"\"
    # Convert to DataFrame if dictionary
    if isinstance(raw_data, dict):
        data = pd.DataFrame([raw_data])
    else:
        data = raw_data.copy()

    # Required features for the model
    required_features = {X_train.columns.tolist()}

    # Fill missing values
    for feature in required_features:
        if feature not in data.columns:
            data[feature] = 0  # Default value

    # Select and order features
    X = data[required_features]

    # Fill any remaining missing values
    for col in X.columns:
        if X[col].isna().any():
            X[col] = X[col].fillna(safe_median(X[col]) if pd.api.types.is_numeric_dtype(X[col]) else 'UNKNOWN')

    # Apply scaling
    means = {scaler.mean_.tolist() if hasattr(scaler, 'mean_') else [0]*len(X_train.columns)}
    stds = {scaler.scale_.tolist() if hasattr(scaler, 'scale_') else [1]*len(X_train.columns)}

    # Apply scaling with error handling
    X_scaled = np.zeros(X.shape)
    for i, col in enumerate(X.columns):
        try:
            X_scaled[:, i] = (X[col].values - means[i]) / max(stds[i], 1e-10)  # Avoid division by zero
        except Exception as e:
            print(f"Error scaling column {{col}}: {{str(e)}}")
            X_scaled[:, i] = X[col].values  # Use unscaled values as fallback

    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    return X_scaled_df

# Example usage:
# import pickle
# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)
#
# sample_data = {{
#     'community_area': '1',
#     'month': 1,
#     'year': 2024,
#     # Add other required feature values
# }}
#
# processed = preprocess_for_prediction(sample_data)
# prediction = model.predict(processed)
# print(f"Predicted {TARGET}: {{prediction[0]:.2f}}")
"""

                # Save preprocessing code
                preproc_path = f"{MODELS_DIR}/preprocessing_{datetime.now():%Y%m%d_%H%M%S}.py"
                with open(preproc_path, 'w') as f:
                    f.write(preprocessing_code)
                log(f"Saved preprocessing code to {preproc_path}")

                # Create simple example prediction script
                if best_model_name in trained_models:
                    model_path = f"{MODELS_DIR}/{best_model_name.lower().replace(' ', '_')}_{datetime.now():%Y%m%d_%H%M%S}.pkl"
                    prediction_code = f"""
# Example prediction script
import pickle
import pandas as pd
import numpy as np
import json

# Load the model
model_path = "{model_path}"  # Update this path to where your model is saved

# Load model with error handling
try:
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
except Exception as e:
    print(f"Error loading model: {{str(e)}}")
    exit(1)

# Import preprocessing function
from preprocessing_{datetime.now():%Y%m%d_%H%M%S} import preprocess_for_prediction

def predict_crime(input_data):
    \"\"\"
    Predict crime for the given input data

    Parameters:
    -----------
    input_data : dict or DataFrame
        Input data with community area and temporal features

    Returns:
    --------
    float
        Predicted crime {TARGET} value
    \"\"\"
    try:
        # Preprocess the data
        processed = preprocess_for_prediction(input_data)

        # Make prediction
        prediction = model.predict(processed)

        return float(prediction[0])
    except Exception as e:
        print(f"Error making prediction: {{str(e)}}")
        return None

# Example usage
if __name__ == "__main__":
    # Example data for a community area in a specific month
    sample_data = {{
        'month': 1,
        'year': 2024,
        'community_area': '1',
        # Add values for required features
    }}

    result = predict_crime(sample_data)
    if result is not None:
        print(f"Predicted {TARGET}: {{result:.2f}}")
    else:
        print("Prediction failed")
"""

                    # Save prediction script
                    predict_path = f"{MODELS_DIR}/predict_{datetime.now():%Y%m%d_%H%M%S}.py"
                    with open(predict_path, 'w') as f:
                        f.write(prediction_code)
                    log(f"Saved prediction script to {predict_path}")
                else:
                    log("No best model available for creating prediction script")

        except Exception as e:
            log(f"Error preparing deployment files: {str(e)}")

        # ---------- 10. final summary --------------------------------------
        log("Generating final summary")

        # Create summary of model performance
        summary = f"""
# Improved Predictive Policing Model Summary
Date: {datetime.now():%Y-%m-%d %H:%M:%S}

## Dataset Information
- Target Variable: {TARGET}
- Panel Data Structure: {len(community_areas)} community areas × {12*len(TIME_RANGE)} months
- Time Range: {min(TIME_RANGE)}-{max(TIME_RANGE)}
- Total Records: {len(panel_df):,}
- Features Used: {X_train.shape[1]}
- Fairness Mitigation Applied: {APPLY_FAIRNESS and FAIRLEARN_AVAILABLE}
- Explainability Tools Used: {USE_EXPLAINERS and (SHAP_AVAILABLE or LIME_AVAILABLE)}

## Model Performance
"""

        # Add each model's performance
        for model_name, metrics in model_results.items():
            rmse = metrics.get('rmse', 'N/A')
            r2 = metrics.get('r2', 'N/A')
            mae = metrics.get('mae', 'N/A')

            if not isinstance(rmse, str):
                rmse = f"{rmse:.3f}"
            if not isinstance(r2, str):
                r2 = f"{r2:.3f}"
            if not isinstance(mae, str):
                mae = f"{mae:.3f}"

            summary += f"- {model_name}: RMSE = {rmse}, R² = {r2}, MAE = {mae}\n"

        # Add best model info
        if best_model_name:
            summary += f"\n## Best Model: {best_model_name}\n"

            # Add feature importance for Gradient Boosting model if available
            if best_model_name == 'Gradient Boosting' and 'Gradient Boosting' in trained_models:
                gb_model, _ = trained_models['Gradient Boosting']

                # Extract feature importances
                feature_importances = None
                if hasattr(gb_model, 'feature_importances_'):
                    feature_importances = pd.Series(gb_model.feature_importances_, index=X_train.columns)
                elif hasattr(gb_model, '_estimator') and hasattr(gb_model._estimator, 'feature_importances_'):
                    feature_importances = pd.Series(gb_model._estimator.feature_importances_, index=X_train.columns)
                else:
                    # Use surrogate model as fallback
                    try:
                        surrogate_gb = GradientBoostingRegressor(n_estimators=100, random_state=RAND_STATE)
                        surrogate_gb.fit(X_train_scaled_df, y_train)
                        feature_importances = pd.Series(surrogate_gb.feature_importances_, index=X_train.columns)
                    except Exception as e:
                        log(f"Error creating surrogate model for feature importance: {str(e)}")

                if feature_importances is not None:
                    importances = feature_importances.sort_values(ascending=False)

                    summary += "\n### Most Important Features (Gradient Boosting):\n"
                    for i, (col, imp) in enumerate(importances.items()):
                        if i < 10:  # Show top 10
                            summary += f"{i+1}. {col}: {imp:.4f}\n"

            # Add Ridge coefficients if available
            if 'Ridge' in trained_models:
                ridge_pipeline, _ = trained_models['Ridge']
                ridge_model = ridge_pipeline.named_steps.get('ridge')

                if ridge_model and hasattr(ridge_model, 'coef_'):
                    coef = pd.Series(ridge_model.coef_, index=X_train.columns)
                    coef = coef.reindex(coef.abs().sort_values(ascending=False).index)

                    summary += "\n### Most Important Features (Ridge Coefficients):\n"
                    for i, (col, coef_val) in enumerate(coef.items()):
                        if i < 10:  # Show top 10
                            summary += f"{i+1}. {col}: {coef_val:.4f}\n"

        # Add fairness analysis results if available
        if fairness_results:
            summary += "\n## Fairness Analysis\n"

            for attr_name, attr_results in fairness_results.items():
                summary += f"\n### Protected Attribute: {attr_name}\n"

                # Handle different result formats
                if 'disparities' in attr_results:
                    summary += "Disparity Metrics:\n"
                    for metric, value in attr_results['disparities'].items():
                        if not isinstance(value, str) and not np.isnan(value):
                            summary += f"- {metric}: {value:.3f}\n"
                elif isinstance(attr_results, dict):
                    summary += "Disparity Metrics:\n"
                    for metric, value in attr_results.items():
                        if not isinstance(value, str) and not np.isnan(value):
                            summary += f"- {metric}: {value:.3f}\n"

        # Add ethical considerations
        summary += """
## Ethical Considerations
- The model predictions should be interpreted as risk factors, not as deterministic outcomes.
- Fairness analysis should be regularly repeated to ensure the model does not systematically disadvantage protected groups.
- Deployment should include human oversight and routine auditing for bias or unintended consequences.
- Transparency about the model's limitations and confidence intervals should be maintained.
- The model should be periodically retrained as socioeconomic conditions and crime patterns evolve.

## Next Steps & Recommendations
1. Incorporate additional datasets (weather, economic indicators, etc.) to improve prediction accuracy.
2. Explore more sophisticated fairness mitigation techniques.
3. Develop an interactive dashboard for interpretable results by non-technical stakeholders.
4. Establish a continuous monitoring system for model drift and bias detection.
5. Conduct community engagement to ensure the system aligns with community needs and priorities.
"""

        # Save summary to file
        summary_path = f"{MODELS_DIR}/model_summary_{datetime.now():%Y%m%d_%H%M%S}.md"
        with open(summary_path, 'w') as f:
            f.write(summary)
        log(f"Saved model summary to {summary_path}")

        log("Analysis complete!")

        return {
            'model_results': model_results,
            'best_model': best_model_name,
            'fairness_results': fairness_results,
            'summary_path': summary_path
        }
    else:
        log("Panel data approach not used. Implement alternative modeling here if needed.")
        return {}

if __name__ == "__main__":
    try:
        log("Starting predictive policing analysis")
        results = main()
        log("Analysis completed successfully")
    except Exception as e:
        log(f"Critical error in main execution: {str(e)}")
        import traceback
        log(traceback.format_exc())

Fairlearn not available. Install with: pip install fairlearn
18:05:44 | Starting predictive policing analysis
18:05:50 | Loading cached crime data
18:05:50 | Loading cached community data
18:05:50 | Loading cached census data
18:05:50 | Loading cached boundary data
18:05:51 | Applied year filter [2022, 2023, 2024]: 760,811 rows (from 2,000,000)
18:05:51 | Data quality check for Crime dataset:
18:05:51 |   Shape: (760811, 31)
18:05:52 |   Missing values: 119946
18:05:52 |   Columns: id, case_number, date, block, iucr, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on, latitude, longitude, location, :@computed_region_awaf_s7ux, :@computed_region_6mkv_f3dw, :@computed_region_vrxf_vc4k, :@computed_region_bdys_3d7i, :@computed_region_43wa_7qmu, :@computed_region_rpca_8um6, :@computed_region_d9mm_jgwp, :@computed_region_d3ds_rm58, :@computed_region_8hcu_yrd4
18:05:53 | Data quality c

[I 2025-05-02 18:07:34,753] A new study created in memory with name: no-name-b135333a-97e7-4457-867a-e12e7a2d252a
[I 2025-05-02 18:07:34,764] Trial 0 finished with value: 205.52245448432836 and parameters: {'alpha': 0.34077205250558684}. Best is trial 0 with value: 205.52245448432836.
[I 2025-05-02 18:07:34,773] Trial 1 finished with value: 205.49649654005714 and parameters: {'alpha': 63.32653429889588}. Best is trial 1 with value: 205.49649654005714.
[I 2025-05-02 18:07:34,783] Trial 2 finished with value: 205.52250643482114 and parameters: {'alpha': 0.3296252095932128}. Best is trial 1 with value: 205.49649654005714.
[I 2025-05-02 18:07:34,792] Trial 3 finished with value: 205.5109348747338 and parameters: {'alpha': 2.931392055633154}. Best is trial 1 with value: 205.49649654005714.
[I 2025-05-02 18:07:34,801] Trial 4 finished with value: 205.52024205181064 and parameters: {'alpha': 0.8196562305184616}. Best is trial 1 with value: 205.49649654005714.
[I 2025-05-02 18:07:34,810] Trial

18:07:34 | Excluding percent_aged_16_unemployed (corr=0.806) in favor of percent_households_below_poverty (corr=0.154)
18:07:34 | Excluding hardship_index (corr=0.805) in favor of percent_households_below_poverty (corr=0.154)
18:07:34 | Excluded 8 features with high correlation
18:07:34 | Keeping 43 features
18:07:34 | Panel data shape after correlation filtering: (2808, 43)
18:07:34 | Train data: (936, 44) rows from 2022
18:07:34 | Validation data: (936, 44) rows from 2023
18:07:34 | Test data: (936, 44) rows from 2024
18:07:34 | Selecting up to 30 important features
18:07:34 | Selected 21 features with variance > 0.01
18:07:34 | Final feature set: 21 features
18:07:34 | Imputing missing values (before: 36 NaNs)
18:07:34 |   Imputed 12 missing values in 'percent_households_below_poverty' with median=18.900
18:07:34 |   Imputed 12 missing values in 'percent_aged_25_without_high_school_diploma' with median=18.800
18:07:34 |   Imputed 12 missing values in 'percent_aged_under_18_or_over_6

[I 2025-05-02 18:07:34,901] Trial 13 finished with value: 205.56335857161022 and parameters: {'alpha': 81.25605744665289}. Best is trial 11 with value: 205.4728939838226.
[I 2025-05-02 18:07:34,914] Trial 14 finished with value: 205.47192048817834 and parameters: {'alpha': 14.661599177666869}. Best is trial 14 with value: 205.47192048817834.
[I 2025-05-02 18:07:34,928] Trial 15 finished with value: 205.51330155081055 and parameters: {'alpha': 2.378472854063141}. Best is trial 14 with value: 205.47192048817834.
[I 2025-05-02 18:07:34,941] Trial 16 finished with value: 205.449860007722 and parameters: {'alpha': 31.08276131594949}. Best is trial 16 with value: 205.449860007722.
[I 2025-05-02 18:07:34,955] Trial 17 finished with value: 205.44949459063412 and parameters: {'alpha': 33.50181864900946}. Best is trial 17 with value: 205.44949459063412.
[I 2025-05-02 18:07:34,969] Trial 18 finished with value: 205.52399248845052 and parameters: {'alpha': 0.012621349178344173}. Best is trial 17 w

18:07:34 | Best Ridge alpha: 33.5018
18:07:34 | Ridge - RMSE: 213.208, R²: 0.112, MAE: 155.876
18:07:34 | Top Ridge coefficients:
18:07:34 |   1. rolling_std_3m: 87.0713
18:07:34 |   2. percent_households_below_poverty: 46.2021
18:07:34 |   3. month_2: 41.3476
18:07:34 |   4. percent_aged_25_without_high_school_diploma: 23.0141
18:07:34 |   5. percent_aged_under_18_or_over_64: 18.9821
18:07:34 | Training Gradient Boosting model


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>